<a href="https://colab.research.google.com/github/tivanello/fase5/blob/main/notebooks/Entrega%20DATATHON%20Fase5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Coleta dos dados**

In [84]:
###############################################################################################################################################
# Importação das abas (PEDE2022, PEDE2023, PEDE2024) e criação do df_fase5
#
# O que eu faço aqui:
# - Mont0 a URL do arquivo Excel no GitHub (raw) tratando espaços e caracteres especiais no nome do arquivo.
# - Abro o Excel remoto e listo as abas disponíveis (checagem rápida de nomes).
# - Leio as três abas (PEDE2022, PEDE2023, PEDE2024) em dataframes separados.
# - Crio a coluna ano_base em cada aba para rastrear a origem do registro.
# - Concatêno tudo em um único dataframe (df_fase5) para EDA e modelagem.
# - Exibo shapes e uma amostra inicial para validar que a carga ficou ok.
###############################################################################################################################################

import pandas as pd
from urllib.parse import quote

base = "https://raw.githubusercontent.com/tivanello/fase5/main/data/raw/"
nome = "BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

url = base + quote(nome)

# Conferir abas
xls = pd.ExcelFile(url)
print("Abas encontradas:", xls.sheet_names)

# Ler abas (ajuste se tiver diferença de maiúsculas/minúsculas)
df_2022 = pd.read_excel(url, sheet_name="PEDE2022")
df_2023 = pd.read_excel(url, sheet_name="PEDE2023")
df_2024 = pd.read_excel(url, sheet_name="PEDE2024")

# Tag de ano (pra não virar bagunça depois)
df_2022["ano_base"] = 2022
df_2023["ano_base"] = 2023
df_2024["ano_base"] = 2024

# Junta tudo
df_fase5 = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

print("Shapes:", df_2022.shape, df_2023.shape, df_2024.shape)
print("df_fase5 shape:", df_fase5.shape)
display(df_fase5.head())



Abas encontradas: ['PEDE2022', 'PEDE2023', 'PEDE2024']
Shapes: (860, 43) (1014, 49) (1156, 51)
df_fase5 shape: (3030, 64)


,RA,Fase,Turma,Nome,Ano nasc,Idade 22,Gênero,Ano ingresso,Instituição de ensino,Pedra 20,...,Fase Ideal,Defasagem,Destaque IPV.1,INDE 2024,Pedra 2024,Avaliador5,Avaliador6,Escola,Ativo/ Inativo,Ativo/ Inativo.1
0,RA-1,7,A,Aluno-1,2003.0,19.0,Menina,2016,Escola Pública,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RA-2,7,A,Aluno-2,2005.0,17.0,Menina,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RA-3,7,A,Aluno-3,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RA-4,7,A,Aluno-4,2005.0,17.0,Menino,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RA-5,7,A,Aluno-5,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df_fase5.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 64 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RA                     3030 non-null   object 
 1   Fase                   3030 non-null   object 
 2   Turma                  3030 non-null   object 
 3   Nome                   860 non-null    object 
 4   Ano nasc               860 non-null    float64
 5   Idade 22               860 non-null    float64
 6   Gênero                 3030 non-null   object 
 7   Ano ingresso           3030 non-null   int64  
 8   Instituição de ensino  3029 non-null   object 
 9   Pedra 20               754 non-null    object 
 10  Pedra 21               1061 non-null   object 
 11  Pedra 22               1932 non-null   object 
 12  INDE 22                1932 non-null   float64
 13  Cg                     860 non-null    float64
 14  Cf                     860 non-null    float64
 15  Ct  

In [86]:
###############################################################################################################################################
# BLOCO: Conferência pós-importação (sanidade do df_fase5)
#
# O que eu faço aqui:
# - Confirmo se a concatenação das abas ficou consistente (quantidade de linhas por ano_base).
# - Confiro quantos alunos únicos (RA) existem por ano_base.
# - Verifico se há RA duplicado dentro do mesmo ano (isso atrapalha EDA/modelagem).
# - Identifico colunas 100% vazias (lixo de exportação do Excel).
# - Aponto colunas “suspeitas” que estão como texto (object), mas deveriam ser número ou data,
#   e mostro uma amostra dos valores para entender o padrão antes de corrigir.
###############################################################################################################################################

# 1) Linhas por ano + RAs únicos
print("Linhas por ano_base:")
print(df_fase5["ano_base"].value_counts(dropna=False).sort_index())

print("\nRAs únicos por ano_base:")
print(df_fase5.groupby("ano_base")["RA"].nunique())

# 2) Duplicidade de RA por ano
dup = df_fase5.duplicated(subset=["ano_base", "RA"], keep=False)
print("\nLinhas com RA duplicado dentro do mesmo ano:", int(dup.sum()))
if dup.any():
    display(df_fase5.loc[dup, ["ano_base", "RA", "Turma"]].sort_values(["ano_base","RA"]).head(20))

# 3) Colunas 100% vazias
zero_non_null = [c for c in df_fase5.columns if df_fase5[c].notna().sum() == 0]

print("Qtd de colunas 100% vazias:", len(zero_non_null))
print("Colunas 100% vazias:")
for c in zero_non_null:
    print("-", c)

# 4) Tipos suspeitos (object)
suspeitas = [c for c in df_fase5.columns
             if ("INDE" in c or c in ["Idade","Data de Nasc","INDE 2024"])
             and df_fase5[c].dtype == "object"]
print("\nColunas suspeitas (object onde deveria ser numérico/data):", suspeitas)

# 5) Amostra das suspeitas
for c in suspeitas[:6]:
    print("\nAmostra ->", c)
    display(df_fase5[c].dropna().astype(str).head(10))



Linhas por ano_base:
ano_base
2022     860
2023    1014
2024    1156
Name: count, dtype: int64

RAs únicos por ano_base:
ano_base
2022     860
2023    1014
2024    1156
Name: RA, dtype: int64

Linhas com RA duplicado dentro do mesmo ano: 0
Qtd de colunas 100% vazias: 1
Colunas 100% vazias:
- Destaque IPV.1

Colunas suspeitas (object onde deveria ser numérico/data): ['Data de Nasc', 'Idade', 'INDE 2024']

Amostra -> Data de Nasc


,Data de Nasc
860,6/17/2015
861,5/31/2014
862,2/25/2016
863,2015-12-03 00:00:00
864,11/13/2014
865,2016-10-02 00:00:00
866,6/29/2015
867,2015-08-11 00:00:00
868,1/15/2015
869,10/20/2014



Amostra -> Idade


,Idade
860,8
861,9
862,7
863,1900-01-08 00:00:00
864,8
865,1900-01-07 00:00:00
866,8
867,1900-01-07 00:00:00
868,8
869,9



Amostra -> INDE 2024


,INDE 2024
1874,7.611366666700001
1875,8.002866666700001
1876,7.952200000100001
1877,7.156366666600001
1878,5.444199999900001
1879,8.0822
1880,8.959700000000002
1881,7.346677272700001
1882,8.152624242500002
1883,7.982890909100001


### **2. Sanitização / Limpeza / Transformação dos dados**

In [87]:
###############################################################################################################################################
#  Copiar df_fase5 para trabalhar com segurança
#
# O que eu faço aqui:
# - Crio uma cópia do df_fase5 para não correr o risco de estragar o original sem querer.
###############################################################################################################################################

import pandas as pd
import numpy as np

df = df_fase5.copy()
print("OK. Cópia criada. Shape:", df.shape)


OK. Cópia criada. Shape: (3030, 64)


In [88]:
###############################################################################################################################################
#  Remover colunas 100% vazias (0 valores preenchidos)
#
# O que eu faço aqui:
# - Procuro colunas que não têm nenhum valor preenchido (só NaN).
# - Removo essas colunas porque são sobra/lixo do Excel e atrapalham.
###############################################################################################################################################

vazias = [c for c in df.columns if df[c].notna().sum() == 0]
print("Colunas 100% vazias (serão removidas):", vazias)

if vazias:
    df = df.drop(columns=vazias)

print("OK. Shape após remover vazias:", df.shape)


Colunas 100% vazias (serão removidas): ['Destaque IPV.1']
OK. Shape após remover vazias: (3030, 63)


In [89]:
###############################################################################################################################################
#  Ajustar Data de Nasc para datetime
#
# O que eu faço aqui:
# - Converto a coluna Data de Nasc para data (datetime).
# - Funciona mesmo com formatos mistos (ex.: 6/17/2015 e 2015-12-03 00:00:00).
# - Valores inválidos viram NaT (nulo de data).
###############################################################################################################################################

if "Data de Nasc" in df.columns:
    df["Data de Nasc"] = pd.to_datetime(df["Data de Nasc"], errors="coerce")
    print("OK. Data de Nasc ->", df["Data de Nasc"].dtype)
else:
    print("Coluna Data de Nasc não existe no df.")


OK. Data de Nasc -> datetime64[ns]


In [90]:
###############################################################################################################################################
#  Ajustar Idade para inteiro (corrigir erro típico do Excel)
#
# O que eu faço aqui:
# - Tenta converter Idade para número.
# - Se Idade estiver como data (1900-01-xx), eu extraio o 'dia' como idade (ex.: 1900-01-08 -> 8).
# - No final, Idade vira Int64 (inteiro com suporte a nulos).
###############################################################################################################################################

if "Idade" in df.columns:
    s = df["Idade"]

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        idade_dt.notna() & (idade_dt.dt.year == 1900) & (idade_dt.dt.month == 1),
        idade_dt.dt.day,
        np.nan
    )

    df["Idade"] = pd.Series(idade_num).fillna(pd.Series(idade_from_date)).astype("Int64")
    print("OK. Idade ->", df["Idade"].dtype)
else:
    print("Coluna Idade não existe no df.")


OK. Idade -> Int64


In [91]:
###############################################################################################################################################
#  Ajustar INDE 2024 para float
#
# O que eu faço aqui:
# - Converto INDE 2024 para número (float).
# - Se estiver como texto, vira float; se tiver lixo, vira NaN.
###############################################################################################################################################

if "INDE 2024" in df.columns:
    df["INDE 2024"] = pd.to_numeric(df["INDE 2024"], errors="coerce")
    print("OK. INDE 2024 ->", df["INDE 2024"].dtype)
else:
    print("Coluna INDE 2024 não existe no df.")


OK. INDE 2024 -> float64


In [92]:
###############################################################################################################################################
# Conferência final + atualizar df_fase5
#
# O que eu faço aqui:
# - Mostro os tipos finais das colunas críticas.
# - Atualizo o df_fase5 para seguir o fluxo já com os dados corrigidos.
###############################################################################################################################################

for c in ["Data de Nasc", "Idade", "INDE 2024"]:
    if c in df.columns:
        print(c, "->", df[c].dtype)

df_fase5 = df
print("OK. df_fase5 atualizado. Shape:", df_fase5.shape)


Data de Nasc -> datetime64[ns]
Idade -> Int64
INDE 2024 -> float64
OK. df_fase5 atualizado. Shape: (3030, 63)


In [93]:
###############################################################################################################################################
# BLOCO: Auditoria geral de tipos (diagnóstico)
#
# O que eu faço aqui:
# - Examino todas as colunas do df_fase5 e marco como "suspeitas" quando:
#   1) A coluna é texto (object/string), mas parece numérica (muitos valores viram número após limpeza)
#   2) A coluna é texto (object/string) e tem "cara de data" pelo nome (ex.: data, nasc, nascimento),
#      e muitos valores viram datetime
#   3) A coluna é float, mas parece inteiro (quase tudo é inteiro, só ficou float por causa de NaN)
# - Eu só tento parsear data nas colunas que parecem ser de data (pra evitar aviso e custo).
###############################################################################################################################################

import pandas as pd
import numpy as np
import re
import warnings

df = df_fase5.copy()

def clean_numeric_series(s):
    x = s.astype("string").str.strip()
    x = x.str.replace(r"\.", "", regex=True)     # remove pontos (milhar)
    x = x.str.replace(",", ".", regex=False)     # vírgula decimal -> ponto
    x = x.str.replace(r"[^0-9\.\-]", "", regex=True)
    return pd.to_numeric(x, errors="coerce")

def looks_like_date_col(colname):
    c = str(colname).lower()
    return any(k in c for k in ["data", "dt", "nasc", "nascimento", "date"])

def guess_profile(col):
    s = df[col]
    dtype = str(s.dtype)

    sample = s.dropna()
    if len(sample) > 500:
        sample = sample.sample(500, random_state=42)

    pct_num = None
    pct_date = None
    pct_intlike = None

    if dtype in ["object", "string"]:
        num = clean_numeric_series(sample)
        pct_num = float(num.notna().mean()) if len(sample) else 0.0

        # Só tenta data se o nome da coluna indicar que é data
        if looks_like_date_col(col):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                dt = pd.to_datetime(sample.astype("string").str.strip(), errors="coerce")
            pct_date = float(dt.notna().mean()) if len(sample) else 0.0
        else:
            pct_date = 0.0

    if "float" in dtype:
        vals = sample.astype(float)
        pct_intlike = float(np.isclose(vals.dropna() % 1, 0).mean()) if vals.notna().any() else 0.0

    return dtype, pct_num, pct_date, pct_intlike, sample.astype("string").head(8).tolist()

suspeitas = []

for col in df.columns:
    dtype, pct_num, pct_date, pct_intlike, amostra = guess_profile(col)

    flag = False
    motivo = []

    if pct_num is not None and pct_num >= 0.85:
        flag = True
        motivo.append(f"parece_numero={pct_num:.0%}")

    # só marcamos "parece_data" se for coluna com cara de data
    if looks_like_date_col(col) and pct_date is not None and pct_date >= 0.60:
        flag = True
        motivo.append(f"parece_data={pct_date:.0%}")

    if pct_intlike is not None and pct_intlike >= 0.98:
        flag = True
        motivo.append(f"float_parece_inteiro={pct_intlike:.0%}")

    if flag:
        suspeitas.append({
            "coluna": col,
            "dtype_atual": dtype,
            "motivo": "; ".join(motivo),
            "amostra": amostra
        })

suspeitas_df = pd.DataFrame(suspeitas).sort_values(["dtype_atual","coluna"]).reset_index(drop=True)

print("Qtd colunas suspeitas:", len(suspeitas_df))
display(suspeitas_df)


Qtd colunas suspeitas: 21


,coluna,dtype_atual,motivo,amostra
0,Ano nasc,float64,float_parece_inteiro=100%,"[2013.0, 2012.0, 2006.0, 2008.0, 2009.0, 2013...."
1,Cf,float64,float_parece_inteiro=100%,"[166.0, 169.0, 33.0, 140.0, 114.0, 6.0, 75.0, ..."
2,Cg,float64,float_parece_inteiro=100%,"[604.0, 704.0, 413.0, 843.0, 649.0, 32.0, 279...."
3,Ct,float64,float_parece_inteiro=100%,"[9.0, 14.0, 5.0, 7.0, 9.0, 1.0, 4.0, 3.0]"
4,Defas,float64,float_parece_inteiro=100%,"[-1.0, -1.0, -2.0, -1.0, -1.0, -1.0, -1.0, -2.0]"
5,Defasagem,float64,float_parece_inteiro=100%,"[-1.0, -2.0, -1.0, 0.0, -1.0, -1.0, 0.0, -2.0]"
6,IAN,float64,float_parece_inteiro=98%,"[5.0, 5.0, 5.0, 10.0, 5.0, 5.0, 5.0, 5.0]"
7,Idade 22,float64,float_parece_inteiro=100%,"[9.0, 10.0, 16.0, 14.0, 13.0, 9.0, 10.0, 15.0]"
8,Nº Av,float64,float_parece_inteiro=100%,"[3.0, 3.0, 4.0, 0.0, 2.0, 4.0, 2.0, 3.0]"
9,Avaliador1,object,parece_numero=100%,"[Avaliador-21, Avaliador-13, Avaliador-22, Ava..."


In [94]:
###############################################################################################################################################
# Ajuste de tipo da coluna "Ano nasc" (float -> inteiro com NA)
#
# O que eu faço aqui:
# - Converto "Ano nasc" para numérico de forma segura (coerção de erros).
# - Arredondo/normalizo (quando vier como 2013.0) e converto para Int64 (aceita NA).
# - Marco como NA valores fora de um intervalo plausível (evita lixo).
# - Mostro contagem de nulos e amostra antes/depois.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL = "Ano nasc"

print("Antes:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" amostra:", df[COL].dropna().head(10).tolist())

# 1) garante numérico
s = pd.to_numeric(df[COL], errors="coerce")

# 2) como parece inteiro 100%, arredonda e converte
s = s.round(0)

# 3) valida faixa plausível (ajuste se quiser)
#    Exemplo: alunos nascidos entre 1990 e 2020
MIN_ANO = 1990
MAX_ANO = 2020
s = s.where((s >= MIN_ANO) & (s <= MAX_ANO), pd.NA)

# 4) aplica como inteiro com NA
df[COL] = s.astype("Int64")

print("\nDepois:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" amostra:", df[COL].dropna().head(10).tolist())

# se você quiser manter df_fase5 atualizado:
df_fase5 = df


Antes:
 dtype: float64
 nulos: 2170
 amostra: [2003.0, 2005.0, 2005.0, 2005.0, 2005.0, 2004.0, 2004.0, 2002.0, 2004.0, 2004.0]

Depois:
 dtype: Int64
 nulos: 2170
 amostra: [2003, 2005, 2005, 2005, 2005, 2004, 2004, 2002, 2004, 2004]


In [95]:
###############################################################################################################################################
# BLOCO: Ajuste de tipo das colunas de ranking (Cf, Cg, Ct) -> inteiro com NA (Int64)
#
# Contexto (dicionário):
# - CF_2022 = ranking na Fase
# - CG_2022 = ranking Geral
# - CT_2022 = ranking na Turma
#
# O que eu faço aqui:
# - Converto Cf, Cg e Ct para numérico com coerção de erros.
# - Arredondo (remove .0) e converto para Int64 (aceita NA).
# - Valido apenas o mínimo (ranking deve ser >= 1). Não limito teto para não cortar valor válido.
# - Mostro antes/depois.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

cols = ["Cf", "Cg", "Ct"]

for COL in cols:
    if COL not in df.columns:
        print(f"Coluna ausente: {COL} (pulando)")
        continue

    print("\n" + "-"*90)
    print(f"Coluna: {COL}")
    print("Antes -> dtype:", df[COL].dtype, "| nulos:", int(df[COL].isna().sum()))
    print("Amostra:", df[COL].dropna().head(10).tolist())

    s = pd.to_numeric(df[COL], errors="coerce").round(0)

    # ranking válido: >= 1 (0 e negativos viram NA)
    s = s.where(s >= 1, pd.NA)

    df[COL] = s.astype("Int64")

    print("Depois -> dtype:", df[COL].dtype, "| nulos:", int(df[COL].isna().sum()))
    print("Amostra:", df[COL].dropna().head(10).tolist())

df_fase5 = df




------------------------------------------------------------------------------------------
Coluna: Cf
Antes -> dtype: float64 | nulos: 2170
Amostra: [18.0, 8.0, 13.0, 15.0, 6.0, 16.0, 11.0, 21.0, 1.0, 17.0]
Depois -> dtype: Int64 | nulos: 2170
Amostra: [18, 8, 13, 15, 6, 16, 11, 21, 1, 17]

------------------------------------------------------------------------------------------
Coluna: Cg
Antes -> dtype: float64 | nulos: 2170
Amostra: [753.0, 469.0, 629.0, 731.0, 344.0, 745.0, 550.0, 836.0, 113.0, 752.0]
Depois -> dtype: Int64 | nulos: 2170
Amostra: [753, 469, 629, 731, 344, 745, 550, 836, 113, 752]

------------------------------------------------------------------------------------------
Coluna: Ct
Antes -> dtype: float64 | nulos: 2170
Amostra: [10.0, 3.0, 6.0, 7.0, 2.0, 8.0, 5.0, 13.0, 1.0, 9.0]
Depois -> dtype: Int64 | nulos: 2170
Amostra: [10, 3, 6, 7, 2, 8, 5, 13, 1, 9]


In [96]:
###############################################################################################################################################
# Unificar e ajustar tipo de "Defas" e "Defasagem" (mantendo códigos negativos)
#
# O que eu faço aqui:
# - Converto Defas e Defasagem para numérico com coerção de erros.
# - Crio "Defasagem_final":
#     usa Defasagem quando preenchido, senão usa Defas.
# - Identifico conflitos (quando as duas existem e têm valores diferentes).
# - Converto para Int64 (inteiro com NA), mantendo valores negativos (códigos).
# - Mostro distribuição de valores (contagem) pra validar rapidamente.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL_A = "Defas"
COL_B = "Defasagem"
COL_OUT = "Defasagem_final"

# garante que as colunas existam
for c in [COL_A, COL_B]:
    if c not in df.columns:
        print(f"Coluna ausente: {c}")

# converte para numérico (sem matar negativos)
if COL_A in df.columns:
    a = pd.to_numeric(df[COL_A], errors="coerce").round(0)
else:
    a = pd.Series([pd.NA] * len(df), index=df.index, dtype="float64")

if COL_B in df.columns:
    b = pd.to_numeric(df[COL_B], errors="coerce").round(0)
else:
    b = pd.Series([pd.NA] * len(df), index=df.index, dtype="float64")

# conflitos: quando os dois estão preenchidos e são diferentes
mask_conf = a.notna() & b.notna() & (a != b)
print("Conflitos (Defas vs Defasagem):", int(mask_conf.sum()))

if int(mask_conf.sum()) > 0:
    print("\nAmostra de conflitos (até 20):")
    display(df.loc[mask_conf, ["ano_base", COL_A, COL_B]].head(20))

# coluna final: prioriza Defasagem, depois Defas
out = b.where(b.notna(), a)

# converte para Int64 (inteiro com NA) mantendo negativos
df[COL_OUT] = out.astype("Int64")

print("\nResumo Defasagem_final:")
print(" dtype:", df[COL_OUT].dtype)
print(" nulos:", int(df[COL_OUT].isna().sum()))
print(" valores (contagem):")
print(df[COL_OUT].value_counts(dropna=False).sort_index())

# se quiser: manter df_fase5 atualizado
df_fase5 = df


Conflitos (Defas vs Defasagem): 0

Resumo Defasagem_final:
 dtype: Int64
 nulos: 0
 valores (contagem):
Defasagem_final
-5       1
-4       5
-3      39
-2     383
-1    1259
0     1152
1      165
2       24
3        2
Name: count, dtype: Int64


In [97]:
###############################################################################################################################################
#  Sanidade do IAN (tipo numérico e faixa plausível)
#
# O que eu faço aqui:
# - Forço IAN para numérico (float) com coerção de erros.
# - Mostro min/max e valores suspeitos (fora da faixa esperada).
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL = "IAN"

df[COL] = pd.to_numeric(df[COL], errors="coerce")

print("IAN -> dtype:", df[COL].dtype)
print("nulos:", int(df[COL].isna().sum()))
print("min:", df[COL].min(), "| max:", df[COL].max())

# ajuste a faixa se você já souber a regra do indicador
MIN_V = 0
MAX_V = 10

sus = df[(df[COL].notna()) & ((df[COL] < MIN_V) | (df[COL] > MAX_V))]
print("suspeitos fora da faixa 0–10:", len(sus))

if len(sus) > 0:
    display(sus[["ano_base", "RA", COL]].head(20))


IAN -> dtype: float64
nulos: 0
min: 2.5 | max: 10.0
suspeitos fora da faixa 0–10: 0


In [98]:
###############################################################################################################################################
# BLOCO: Ajuste de tipo da coluna "Idade" (float -> inteiro com NA)
#
# O que eu faço aqui:
# - Converto "Idade" para numérico com coerção de erros.
# - Arredondo (remove .0) e converto para Int64 (aceita NA).
# - Valido faixa plausível de idade para evitar lixo.
# - Mostro antes/depois.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL = "Idade"

print("Antes:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" amostra:", df[COL].dropna().head(10).tolist())

s = pd.to_numeric(df[COL], errors="coerce").round(0)

# faixa plausível (ajuste se quiser)
MIN_IDADE = 0
MAX_IDADE = 100
s = s.where((s >= MIN_IDADE) & (s <= MAX_IDADE), pd.NA)

df[COL] = s.astype("Int64")

print("\nDepois:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" amostra:", df[COL].dropna().head(10).tolist())

df_fase5 = df


Antes:
 dtype: Int64
 nulos: 860
 amostra: [8, 9, 7, 8, 8, 7, 8, 7, 8, 9]

Depois:
 dtype: Int64
 nulos: 860
 amostra: [8, 9, 7, 8, 8, 7, 8, 7, 8, 9]


In [99]:
###############################################################################################################################################
# BLOCO: Ajuste de tipo da coluna "Nº Av" (float -> inteiro com NA)
#
# O que eu faço aqui:
# - Converto "Nº Av" para numérico com coerção de erros.
# - Arredondo (remove .0) e converto para Int64 (aceita NA).
# - Valido faixa mínima/máxima (contagem de avaliadores não deve ser negativa).
# - Mostro antes/depois e distribuição de valores.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL = "Nº Av"

print("Antes:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" amostra:", df[COL].dropna().head(10).tolist())

s = pd.to_numeric(df[COL], errors="coerce").round(0)

# contagem válida: >= 0
# teto: se você usa até Avaliador6, faz sentido limitar em 0..6
MIN_V = 0
MAX_V = 6
s = s.where((s >= MIN_V) & (s <= MAX_V), pd.NA)

df[COL] = s.astype("Int64")

print("\nDepois:")
print(" dtype:", df[COL].dtype)
print(" nulos:", int(df[COL].isna().sum()))
print(" valores (contagem):")
print(df[COL].value_counts(dropna=False).sort_index())

df_fase5 = df


Antes:
 dtype: float64
 nulos: 76
 amostra: [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]

Depois:
 dtype: Int64
 nulos: 76
 valores (contagem):
Nº Av
0        127
2        703
3       1100
4        876
5        142
6          6
<NA>      76
Name: count, dtype: Int64


In [100]:
###############################################################################################################################################
# BLOCO: Ajuste das colunas Avaliador* (manter como texto + normalização; opcional: extrair id numérico)
#
# O que eu faço aqui:
# - Identifico todas as colunas que começam com "Avaliador".
# - Normalizo texto: tira espaços, colapsa espaços internos, padroniza traços, padroniza caixa.
# - Converte strings vazias em NA.
# - (Opcional) Cria colunas Avaliador*_id extraindo o número (ex.: "avaliador-21" -> 21).
###############################################################################################################################################

import pandas as pd
import re

df = df_fase5.copy()

avaliador_cols = [c for c in df.columns if str(c).lower().startswith("avaliador")]
print("Colunas Avaliador* encontradas:", avaliador_cols)

def norm_txt(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    if s == "":
        return pd.NA
    s = s.replace("–", "-").replace("—", "-")
    s = s.lower()
    return s

for c in avaliador_cols:
    df[c] = df[c].apply(norm_txt).astype("string")

# opcional: extrair id numérico (deixa comentado se não quiser)
for c in avaliador_cols:
    df[c + "_id"] = (
        df[c]
        .str.extract(r"(\d+)", expand=False)
        .astype("Int64")
    )

# checagem rápida
for c in avaliador_cols:
    print(f"\n{c} -> distintos:", int(df[c].nunique(dropna=True)), "| nulos:", int(df[c].isna().sum()))
    print(" amostra:", df[c].dropna().unique()[:8])

df_fase5 = df


Colunas Avaliador* encontradas: ['Avaliador1', 'Avaliador2', 'Avaliador3', 'Avaliador4', 'Avaliador5', 'Avaliador6']

Avaliador1 -> distintos: 20 | nulos: 203
 amostra: <StringArray>
[ 'avaliador-5',  'avaliador-6',  'avaliador-4',  'avaliador-7',
 'avaliador-11', 'avaliador-13', 'avaliador-19', 'avaliador-15']
Length: 8, dtype: string

Avaliador2 -> distintos: 23 | nulos: 203
 amostra: <StringArray>
['avaliador-27', 'avaliador-30',  'avaliador-2',  'avaliador-3',
  'avaliador-4',  'avaliador-5',  'avaliador-6',  'avaliador-7']
Length: 8, dtype: string

Avaliador3 -> distintos: 21 | nulos: 996
 amostra: <StringArray>
['avaliador-28', 'avaliador-29', 'avaliador-24', 'avaliador-30',
 'avaliador-31', 'avaliador-15',  'avaliador-3',  'avaliador-2']
Length: 8, dtype: string

Avaliador4 -> distintos: 11 | nulos: 1979
 amostra: <StringArray>
['avaliador-31',  'avaliador-8',  'avaliador-2', 'avaliador-15',
  'avaliador-5', 'avaliador-10',  'avaliador-7', 'avaliador-17']
Length: 8, dtype: strin

In [101]:
###############################################################################################################################################
# BLOCO: Padronização de Fase / Fase Ideal (texto) + extração do número da fase (features numéricas)
#
# O que eu faço aqui:
# - Normalizo texto em "Fase", "Fase Ideal" e "Fase ideal" (espaços, traços, caixa).
# - Unifico "Fase Ideal" e "Fase ideal" em uma coluna final: "FaseIdeal_txt".
# - Extraio o número da fase:
#     * "Fase_num" a partir de "Fase" (ex.: "FASE 1", "3", "2L" -> 1, 3, 2)
#     * "FaseIdeal_num" a partir do texto da fase ideal (ex.: "Fase 3 (...)" -> 3)
# - (Opcional) Extraio sufixo de "Fase" quando existir (ex.: "2L" -> "l")
###############################################################################################################################################

import pandas as pd
import re

df = df_fase5.copy()

COL_FASE = "Fase"
COL_FI_A = "Fase Ideal"
COL_FI_B = "Fase ideal"

def norm_txt(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    if s == "":
        return pd.NA
    s = s.replace("–", "-").replace("—", "-")
    return s

# 1) normaliza as colunas se existirem
if COL_FASE in df.columns:
    df[COL_FASE] = df[COL_FASE].apply(norm_txt).astype("string")

if COL_FI_A in df.columns:
    df[COL_FI_A] = df[COL_FI_A].apply(norm_txt).astype("string")

if COL_FI_B in df.columns:
    df[COL_FI_B] = df[COL_FI_B].apply(norm_txt).astype("string")

# 2) unifica Fase Ideal (prioriza "Fase Ideal", senão usa "Fase ideal")
if (COL_FI_A in df.columns) and (COL_FI_B in df.columns):
    df["FaseIdeal_txt"] = df[COL_FI_A].where(df[COL_FI_A].notna(), df[COL_FI_B])
elif COL_FI_A in df.columns:
    df["FaseIdeal_txt"] = df[COL_FI_A]
elif COL_FI_B in df.columns:
    df["FaseIdeal_txt"] = df[COL_FI_B]
else:
    df["FaseIdeal_txt"] = pd.NA

df["FaseIdeal_txt"] = df["FaseIdeal_txt"].astype("string")

# 3) extrai número da fase (primeiro dígito/grupo numérico)
if COL_FASE in df.columns:
    df["Fase_num"] = df[COL_FASE].str.extract(r"(\d+)", expand=False).astype("Int64")
    # (opcional) sufixo tipo "L" em "2L"
    df["Fase_sufixo"] = df[COL_FASE].str.extract(r"\d+\s*([A-Za-z])", expand=False).str.lower().astype("string")
else:
    df["Fase_num"] = pd.Series([pd.NA]*len(df), index=df.index, dtype="Int64")
    df["Fase_sufixo"] = pd.Series([pd.NA]*len(df), index=df.index, dtype="string")

df["FaseIdeal_num"] = df["FaseIdeal_txt"].str.extract(r"(\d+)", expand=False).astype("Int64")

# 4) checagens rápidas
print("Fase - valores distintos (amostra):", df[COL_FASE].dropna().unique()[:12] if COL_FASE in df.columns else "sem coluna")
print("Fase_num - distintos:", int(df["Fase_num"].nunique(dropna=True)), "| nulos:", int(df["Fase_num"].isna().sum()))
print("Fase_sufixo - distintos:", df["Fase_sufixo"].dropna().unique()[:12])
print("FaseIdeal_num - distintos:", int(df["FaseIdeal_num"].nunique(dropna=True)), "| nulos:", int(df["FaseIdeal_num"].isna().sum()))

# (opcional) se quiser dropar as colunas duplicadas originais depois de validar:
# df = df.drop(columns=[c for c in [COL_FI_A, COL_FI_B] if c in df.columns])

df_fase5 = df


Fase - valores distintos (amostra): <StringArray>
['7', '6', '5', '4', '3', '2', '1', '0', 'ALFA', 'FASE 1', 'FASE 2', 'FASE 3']
Length: 12, dtype: string
Fase_num - distintos: 10 | nulos: 427
Fase_sufixo - distintos: <StringArray>
['a', 'b', 'c', 'd', 'e', 'g', 'h', 'j', 'k', 'l', 'm', 'n']
Length: 12, dtype: string
FaseIdeal_num - distintos: 8 | nulos: 0


In [102]:
###############################################################################################################################################
# BLOCO: Ajuste de "Nome" e "Nome Anonimizado" (texto) + extração opcional de ID numérico
#
# O que eu faço aqui:
# - Normalizo "Nome" e "Nome Anonimizado" como texto (strip, espaços, traços, caixa).
# - Troco strings vazias por NA.
# - (Opcional) Extraio o número do padrão "Aluno-####" para colunas *_id (Int64).
###############################################################################################################################################

import pandas as pd
import re

df = df_fase5.copy()

COL_NOME = "Nome"
COL_ANON = "Nome Anonimizado"

def norm_txt(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    if s == "":
        return pd.NA
    s = s.replace("–", "-").replace("—", "-")
    s = s.lower()
    return s

for col in [COL_NOME, COL_ANON]:
    if col in df.columns:
        df[col] = df[col].apply(norm_txt).astype("string")
    else:
        print(f"Coluna ausente: {col}")

# opcional: extrair id numérico
if COL_NOME in df.columns:
    df["Nome_id"] = df[COL_NOME].str.extract(r"(\d+)", expand=False).astype("Int64")

if COL_ANON in df.columns:
    df["NomeAnon_id"] = df[COL_ANON].str.extract(r"(\d+)", expand=False).astype("Int64")

# checagens rápidas
if COL_NOME in df.columns:
    print("\nNome -> distintos:", int(df[COL_NOME].nunique(dropna=True)), "| nulos:", int(df[COL_NOME].isna().sum()))
    print("amostra:", df[COL_NOME].dropna().unique()[:8])

if COL_ANON in df.columns:
    print("\nNome Anonimizado -> distintos:", int(df[COL_ANON].nunique(dropna=True)), "| nulos:", int(df[COL_ANON].isna().sum()))
    print("amostra:", df[COL_ANON].dropna().unique()[:8])

df_fase5 = df



Nome -> distintos: 860 | nulos: 2170
amostra: <StringArray>
['aluno-1', 'aluno-2', 'aluno-3', 'aluno-4', 'aluno-5', 'aluno-6', 'aluno-7',
 'aluno-8']
Length: 8, dtype: string

Nome Anonimizado -> distintos: 1405 | nulos: 860
amostra: <StringArray>
['aluno-861', 'aluno-862', 'aluno-863', 'aluno-864', 'aluno-865', 'aluno-866',
 'aluno-867', 'aluno-868']
Length: 8, dtype: string


In [103]:
###############################################################################################################################################
# BLOCO: Extrair número do RA (RA-####) para "ra_num" (Int64)
#
# O que eu faço aqui:
# - Encontro a coluna de RA (RA/ra) sem depender de snake_case.
# - Normalizo o texto (strip, espaços, traços) antes de extrair.
# - Extraio o número do padrão RA-#### e salvo como Int64 (aceita NA).
# - Mostro amostra e checo possíveis inconsistências (mapeamento 1:1).
###############################################################################################################################################

import pandas as pd
import re

df = df_fase5.copy()

# acha a coluna RA (independente de maiúscula/minúscula)
ra_candidates = [c for c in df.columns if str(c).strip().lower() == "ra"]
if not ra_candidates:
    raise ValueError("Não encontrei coluna RA/ra no df.")
base_col = ra_candidates[0]

# normaliza texto
s = df[base_col].astype("string")
s = s.str.strip().str.replace(r"\s+", " ", regex=True)
s = s.str.replace("–", "-", regex=False).str.replace("—", "-", regex=False)

# extrai número
df["ra_num"] = pd.to_numeric(
    s.str.extract(r"(\d+)", expand=False),
    errors="coerce"
).astype("Int64")

print("OK. ra_num criado.")
print("Coluna base:", base_col)
print("Nulos em ra_num:", int(df["ra_num"].isna().sum()))
display(df[[base_col, "ra_num"]].drop_duplicates().head(15))

# checagem rápida: 1 RA textual -> 1 ra_num
tmp = df[[base_col, "ra_num"]].dropna().drop_duplicates()
dup_txt = tmp.groupby(base_col)["ra_num"].nunique().sort_values(ascending=False)
dup_num = tmp.groupby("ra_num")[base_col].nunique().sort_values(ascending=False)

print("\nRA textual com mais de 1 ra_num (top 5):")
print(dup_txt[dup_txt > 1].head(5))

print("\nra_num com mais de 1 RA textual (top 5):")
print(dup_num[dup_num > 1].head(5))

df_fase5 = df


OK. ra_num criado.
Coluna base: RA
Nulos em ra_num: 0


,RA,ra_num
0,RA-1,1
1,RA-2,2
2,RA-3,3
3,RA-4,4
4,RA-5,5
5,RA-6,6
6,RA-7,7
7,RA-8,8
8,RA-9,9
9,RA-10,10



RA textual com mais de 1 ra_num (top 5):
Series([], Name: ra_num, dtype: int64)

ra_num com mais de 1 RA textual (top 5):
Series([], Name: RA, dtype: int64)


In [104]:
###############################################################################################################################################
# Verificar se ra_num ficou único dentro de cada ano_base
#
# O que eu faço aqui:
# - Confiro se ra_num é único por ano_base (igual você fez com ra).
# - Se não for, eu mostro exemplos (pra entender se existe reaproveitamento de RA em outro contexto).
###############################################################################################################################################

df = df_fase5.copy()

if "ano_base" in df.columns and "ra_num" in df.columns:
    dup = df.duplicated(subset=["ano_base", "ra_num"], keep=False)
    print("Linhas com ra_num duplicado dentro do mesmo ano:", int(dup.sum()))
    if dup.any():
        cols_show = [c for c in ["ano_base", "ra", "RA", "ra_num", "turma", "Turma"] if c in df.columns]
        display(df.loc[dup, cols_show].sort_values(["ano_base","ra_num"]).head(30))
else:
    print("Faltando ano_base ou ra_num no df.")


Linhas com ra_num duplicado dentro do mesmo ano: 0


In [105]:
df_fase5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 77 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RA                     3030 non-null   object        
 1   Fase                   3030 non-null   string        
 2   Turma                  3030 non-null   object        
 3   Nome                   860 non-null    string        
 4   Ano nasc               860 non-null    Int64         
 5   Idade 22               860 non-null    float64       
 6   Gênero                 3030 non-null   object        
 7   Ano ingresso           3030 non-null   int64         
 8   Instituição de ensino  3029 non-null   object        
 9   Pedra 20               754 non-null    object        
 10  Pedra 21               1061 non-null   object        
 11  Pedra 22               1932 non-null   object        
 12  INDE 22                1932 non-null   float64       
 13  Cg 

In [106]:
###############################################################################################################################################
# BLOCO: Ajuste "Idade 22" (float -> Int64)
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

COL = "Idade 22"
if COL in df.columns:
    s = pd.to_numeric(df[COL], errors="coerce").round(0)
    s = s.where((s >= 0) & (s <= 100), pd.NA)
    df[COL] = s.astype("Int64")

print("dtype Idade 22:", df["Idade 22"].dtype if "Idade 22" in df.columns else "coluna ausente")
df_fase5 = df


dtype Idade 22: Int64


In [107]:
###############################################################################################################################################
# BLOCO: Checagem de duplicidade entre "Ativo/ Inativo" e "Ativo/ Inativo.1"
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

c1 = "Ativo/ Inativo"
c2 = "Ativo/ Inativo.1"

if c1 in df.columns and c2 in df.columns:
    eq = df[c1].astype("string").fillna("<NA>") == df[c2].astype("string").fillna("<NA>")
    print("Linhas iguais (%):", round(100 * eq.mean(), 2))
    print("Divergências:", int((~eq).sum()))
    if int((~eq).sum()) > 0:
        display(df.loc[~eq, ["ano_base", "RA", c1, c2]].head(20))
else:
    print("Uma das colunas não existe.")


Linhas iguais (%): 100.0
Divergências: 0


In [108]:
###############################################################################################################################################
# BLOCO: Remover coluna duplicada "Ativo/ Inativo.1" (idêntica a "Ativo/ Inativo")
###############################################################################################################################################

df = df_fase5.copy()

dup = "Ativo/ Inativo.1"
if dup in df.columns:
    df = df.drop(columns=[dup])
    print("OK. Removida:", dup)

df_fase5 = df


OK. Removida: Ativo/ Inativo.1


In [109]:
###############################################################################################################################################
# BLOCO: Normalização básica de colunas categóricas (strip + espaços)
###############################################################################################################################################

import pandas as pd
import re

df = df_fase5.copy()

cat_cols = [
    "Gênero", "Instituição de ensino", "Turma", "Escola",
    "Ativo/ Inativo", "Ativo/ Inativo.1",
    "Indicado", "Atingiu PV",
    "Destaque IEG", "Destaque IDA", "Destaque IPV",
    "Rec Av1", "Rec Av2", "Rec Av3", "Rec Av4", "Rec Psicologia",
    "Pedra 20", "Pedra 21", "Pedra 22", "Pedra 23", "Pedra 2023", "Pedra 2024"
]

def norm_cat(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return pd.NA if s == "" else s

for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].apply(norm_cat).astype("string")

df_fase5 = df
print("OK. Categorias normalizadas (strip/espaços).")


OK. Categorias normalizadas (strip/espaços).


In [110]:
###############################################################################################################################################
# BLOCO: Checagem de consistência (ano_base vs Ano nasc / Data de Nasc)
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# 2022: Ano nasc x Idade 22
if "Ano nasc" in df.columns and "Idade 22" in df.columns:
    calc_22 = df["ano_base"].astype("Int64") - df["Ano nasc"].astype("Int64")
    m = df["Idade 22"].notna() & df["Ano nasc"].notna()
    dif = (df.loc[m, "Idade 22"].astype("Int64") - calc_22.loc[m]).abs()
    print("Idade 22 vs (ano_base - Ano nasc) | divergência > 1 ano:", int((dif > 1).sum()))
    if int((dif > 1).sum()) > 0:
        display(df.loc[m].assign(calc=calc_22)[dif > 1][["ano_base","RA","Ano nasc","Idade 22","calc"]].head(20))

# 2023/2024: Data de Nasc x Idade
if "Data de Nasc" in df.columns and "Idade" in df.columns:
    calc = df["ano_base"].astype("Int64") - df["Data de Nasc"].dt.year.astype("Int64")
    m = df["Idade"].notna() & df["Data de Nasc"].notna()
    dif = (df.loc[m, "Idade"].astype("Int64") - calc.loc[m]).abs()
    print("Idade vs (ano_base - ano(Data de Nasc)) | divergência > 1 ano:", int((dif > 1).sum()))
    if int((dif > 1).sum()) > 0:
        display(df.loc[m].assign(calc=calc)[dif > 1][["ano_base","RA","Data de Nasc","Idade","calc"]].head(20))


Idade 22 vs (ano_base - Ano nasc) | divergência > 1 ano: 0
Idade vs (ano_base - ano(Data de Nasc)) | divergência > 1 ano: 0


In [111]:
###############################################################################################################################################
# BLOCO: Consolidar INDE e PEDRA por ano_base (colunas finais únicas)
#
# O que eu faço aqui:
# - Crio INDE_final e PEDRA_final pegando o valor correspondente ao ano_base.
# - Registro qual coluna foi usada (INDE_fonte / PEDRA_fonte) para auditoria.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# mapeie possíveis nomes por ano
map_inde = {
    2022: ["INDE 22", "INDE_2022", "INDE 2022"],
    2023: ["INDE 23", "INDE 2023", "INDE_2023", "INDE 2023"],
    2024: ["INDE 2024", "INDE_2024", "INDE 24", "INDE 2024"],
}

map_pedra = {
    2022: ["Pedra 22", "Pedra_2022", "Pedra 2022"],
    2023: ["Pedra 23", "Pedra 2023", "Pedra_2023", "Pedra 2023"],
    2024: ["Pedra 2024", "Pedra_2024", "Pedra 24", "Pedra 2024"],
}

def pick_value(row, candidates):
    for c in candidates:
        if c in row.index and pd.notna(row[c]):
            return row[c], c
    return pd.NA, pd.NA

inde_vals = []
inde_src  = []
ped_vals  = []
ped_src   = []

for _, row in df.iterrows():
    ano = int(row["ano_base"])
    v, s = pick_value(row, map_inde.get(ano, []))
    inde_vals.append(v)
    inde_src.append(s)

    v, s = pick_value(row, map_pedra.get(ano, []))
    ped_vals.append(v)
    ped_src.append(s)

df["INDE_final"] = pd.to_numeric(pd.Series(inde_vals, index=df.index), errors="coerce")
df["INDE_fonte"] = pd.Series(inde_src, index=df.index).astype("string")

df["PEDRA_final"] = pd.Series(ped_vals, index=df.index).astype("string")
df["PEDRA_fonte"] = pd.Series(ped_src, index=df.index).astype("string")

print("INDE_final nulos:", int(df["INDE_final"].isna().sum()))
print("PEDRA_final nulos:", int(df["PEDRA_final"].isna().sum()))
print("Fontes INDE (top):")
print(df["INDE_fonte"].value_counts(dropna=False).head(10))
print("Fontes PEDRA (top):")
print(df["PEDRA_fonte"].value_counts(dropna=False).head(10))

df_fase5 = df


INDE_final nulos: 185
PEDRA_final nulos: 147
Fontes INDE (top):
INDE_fonte
INDE 2024    1054
INDE 2023     931
INDE 22       860
<NA>          185
Name: count, dtype: Int64
Fontes PEDRA (top):
PEDRA_fonte
Pedra 2024    1092
Pedra 2023     931
Pedra 22       860
<NA>           147
Name: count, dtype: Int64


In [112]:
###############################################################################################################################################
# BLOCO: Padronizar rótulos das colunas fonte (INDE_fonte / PEDRA_fonte)
#
# O que eu faço aqui:
# - Troco "INDE 22" -> "INDE 2022"
# - Troco "Pedra 22" -> "Pedra 2022"
# (mantém o resto como está)
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

if "INDE_fonte" in df.columns:
    df["INDE_fonte"] = (
        df["INDE_fonte"].astype("string")
        .str.replace(r"^INDE[\s_]*22$", "INDE 2022", regex=True)
    )

if "PEDRA_fonte" in df.columns:
    df["PEDRA_fonte"] = (
        df["PEDRA_fonte"].astype("string")
        .str.replace(r"^Pedra[\s_]*22$", "Pedra 2022", regex=True)
    )

print("Fontes INDE (top):")
print(df["INDE_fonte"].value_counts(dropna=False).head(10))

print("\nFontes PEDRA (top):")
print(df["PEDRA_fonte"].value_counts(dropna=False).head(10))

df_fase5 = df


Fontes INDE (top):
INDE_fonte
INDE 2024    1054
INDE 2023     931
INDE 2022     860
<NA>          185
Name: count, dtype: Int64

Fontes PEDRA (top):
PEDRA_fonte
Pedra 2024    1092
Pedra 2023     931
Pedra 2022     860
<NA>           147
Name: count, dtype: Int64


In [113]:
###############################################################################################################################################
# BLOCO: Inspeção rápida de conteúdo de colunas (auditoria vs final)
#
# O que eu faço aqui:
# - Mostro dtype, nulos, distintos e amostra de valores (top frequentes + valores únicos).
# - Para colunas numéricas: min/max.
# - Para colunas texto: exemplos de valores.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

cols = [
    "Defas", "Defasagem", "Defasagem_final",
    "Fase Ideal", "Fase ideal", "FaseIdeal_txt",
    "Nome", "Nome Anonimizado", "Nome_id", "NomeAnon_id"
]

def inspeciona_coluna(df, col, top_n=10, amostra_n=12):
    if col not in df.columns:
        print(f"\n{col}: coluna ausente")
        return

    s = df[col]
    print("\n" + "="*100)
    print(f"Coluna: {col}")
    print("dtype:", s.dtype, "| nulos:", int(s.isna().sum()), "| distintos:", int(s.nunique(dropna=True)))

    if pd.api.types.is_numeric_dtype(s):
        print("min:", s.min(), "| max:", s.max())
        print("top valores (contagem):")
        print(s.value_counts(dropna=False).head(top_n))
    else:
        print("top valores (contagem):")
        print(s.astype("string").value_counts(dropna=False).head(top_n))
        print("amostra (valores únicos):")
        vals = s.astype("string").dropna().unique()
        print(vals[:amostra_n])

for c in cols:
    inspeciona_coluna(df, c)



Coluna: Defas
dtype: float64 | nulos: 2170 | distintos: 8
min: -5.0 | max: 2.0
top valores (contagem):
Defas
 NaN    2170
-1.0     410
 0.0     247
-2.0     163
-3.0      23
 1.0       9
-4.0       4
 2.0       3
-5.0       1
Name: count, dtype: int64

Coluna: Defasagem
dtype: float64 | nulos: 860 | distintos: 8
min: -4.0 | max: 3.0
top valores (contagem):
Defasagem
 0.0    905
 NaN    860
-1.0    849
-2.0    220
 1.0    156
 2.0     21
-3.0     16
 3.0      2
-4.0      1
Name: count, dtype: int64

Coluna: Defasagem_final
dtype: Int64 | nulos: 0 | distintos: 9
min: -5 | max: 3
top valores (contagem):
Defasagem_final
-1    1259
0     1152
-2     383
1      165
-3      39
2       24
-4       5
3        2
-5       1
Name: count, dtype: Int64

Coluna: Fase Ideal
dtype: string | nulos: 860 | distintos: 9
top valores (contagem):
Fase Ideal
<NA>                       860
Fase 2 (5° e 6° ano)       527
Fase 3 (7° e 8° ano)       437
Fase 1 (3° e 4° ano)       290
Fase 4 (9° ano)            18

In [114]:
###############################################################################################################################################
# BLOCO: Remover colunas "Defas" e "Defasagem" (manter só "Defasagem_final")
###############################################################################################################################################

df = df_fase5.copy()

cols_drop = [c for c in ["Defas", "Defasagem"] if c in df.columns]
df = df.drop(columns=cols_drop)

print("Removidas:", cols_drop)
print("Colunas restantes relacionadas à defasagem:", [c for c in df.columns if "defas" in str(c).lower()])

df_fase5 = df


Removidas: ['Defas', 'Defasagem']
Colunas restantes relacionadas à defasagem: ['Defasagem_final']


In [115]:
###############################################################################################################################################
# BLOCO: Limpeza das colunas de fase ideal (manter só FaseIdeal_txt + padronizar ° -> º)
#
# O que eu faço aqui:
# - Padronizo o símbolo em FaseIdeal_txt (° -> º) para reduzir duplicidade boba.
# - Apago as colunas originais duplicadas: "Fase Ideal" e "Fase ideal".
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# 1) padroniza símbolo em FaseIdeal_txt
if "FaseIdeal_txt" in df.columns:
    df["FaseIdeal_txt"] = df["FaseIdeal_txt"].astype("string").str.replace("°", "º", regex=False)

# 2) remove duplicadas originais
cols_drop = [c for c in ["Fase Ideal", "Fase ideal"] if c in df.columns]
df = df.drop(columns=cols_drop)

print("Removidas:", cols_drop)

print("OK. FaseIdeal_txt dtype:", df["FaseIdeal_txt"].dtype if "FaseIdeal_txt" in df.columns else "coluna ausente")
print("Distinct FaseIdeal_txt:", int(df["FaseIdeal_txt"].nunique(dropna=True)) if "FaseIdeal_txt" in df.columns else "-")

df_fase5 = df


Removidas: ['Fase Ideal', 'Fase ideal']
OK. FaseIdeal_txt dtype: string
Distinct FaseIdeal_txt: 11


In [116]:
###############################################################################################################################################
# BLOCO: Remover colunas de identificação (Nome / Nome Anonimizado / IDs) para modelagem
#
# O que eu faço aqui:
# - Apago colunas que funcionam como identificador e tendem a causar leakage/memorização no modelo:
#   "Nome", "Nome Anonimizado" e colunas derivadas de ID.
# - Mostro o que foi removido e a nova shape.
###############################################################################################################################################

df = df_fase5.copy()

cols_drop = [
    "Nome", "Nome Anonimizado",
    "Nome_id", "NomeAnon_id",
    "nome_num", "nome_anon_num"
]

cols_drop = [c for c in cols_drop if c in df.columns]

df = df.drop(columns=cols_drop)

print("Removidas (IDs/leakage):", cols_drop)
print("df_fase5 shape (novo):", df.shape)

df_fase5 = df


Removidas (IDs/leakage): ['Nome', 'Nome Anonimizado', 'Nome_id', 'NomeAnon_id']
df_fase5 shape (novo): (3030, 72)


## **Respondendo as perguntas**

* A partir desse ponto serão criadas as respostas para as perguntas do desafio, que em seguida serão exportadas para um XLSX que será lido pelo Power BI para apresentação "História Gerencial"

In [117]:
###############################################################################################################################################
# Indicadores para BI: IAN (faixa) + IDA (ano) + IDA (ano x fase) — sem mexer no df_fase5
#
# O que eu faço aqui:
# - Crio (se não existir) a coluna IAN_faixa com nomes neutros:
#     adequado / atenção / alerta / crítico
# - Crio 3 dataframes em formato longo, prontos para Power BI:
#   1) df_ian      -> distribuição de IAN por faixa e ano_base (qtd e % + ordem)
#   2) df_ida_ano  -> resumo do IDA por ano_base (média, mediana, etc. + delta)
#   3) df_ida_fase -> IDA por ano_base e Fase_num (média/mediana)
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# ----------------------------
# 0) Garantir colunas-base
# ----------------------------
for c in ["IAN", "IDA", "ano_base"]:
    if c not in df.columns:
        raise ValueError(f"Coluna ausente no df: {c}")

df["IAN"] = pd.to_numeric(df["IAN"], errors="coerce")
df["IDA"] = pd.to_numeric(df["IDA"], errors="coerce")

df = df_fase5.copy()

# ----------------------------
# 1) Criar IAN_faixa
# ----------------------------
if "IAN_faixa" not in df.columns:
    def faixa_ian(v):
        if pd.isna(v):
            return pd.NA
        if v >= 8:
            return "adequado"
        if v >= 6:
            return "atenção"
        if v >= 4:
            return "alerta"
        return "crítico"

    df["IAN_faixa"] = df["IAN"].apply(faixa_ian).astype("string")

# ----------------------------
# 2) df_ian - IAN por faixa e ano_base
# ----------------------------
df_ian = (
    df.dropna(subset=["IAN_faixa"])
      .groupby(["ano_base", "IAN_faixa"])
      .size()
      .reset_index(name="qtd")
)

tot = df_ian.groupby("ano_base")["qtd"].transform("sum")
df_ian["pct"] = (df_ian["qtd"] / tot) * 100

ordem = ["adequado", "atenção", "alerta", "crítico"]
df_ian["IAN_faixa_ordem"] = df_ian["IAN_faixa"].map({k:i for i,k in enumerate(ordem, start=1)}).astype("Int64")

# ----------------------------
# 3) df_ida_ano - IDA por ano_base
# ----------------------------
df_ida_ano = (
    df.groupby("ano_base")["IDA"]
      .agg(n="count", ida_media="mean", ida_mediana="median", ida_dp="std", ida_min="min", ida_max="max")
      .reset_index()
      .sort_values("ano_base")
)

df_ida_ano["delta_ida_media"] = df_ida_ano["ida_media"].diff()

# ----------------------------
# 4) df_ida_fase - IDA por ano_base e fase
# ----------------------------
if "Fase_num" in df.columns:
    df_ida_fase = (
        df.groupby(["ano_base", "Fase_num"])["IDA"]
          .agg(n="count", ida_media="mean", ida_mediana="median")
          .reset_index()
          .sort_values(["ano_base", "Fase_num"])
    )
else:
    df_ida_fase = pd.DataFrame(columns=["ano_base", "Fase_num", "n", "ida_media", "ida_mediana"])

print("OK. Dataframes criados:")
print("df_ian:", df_ian.shape)
print("df_ida_ano:", df_ida_ano.shape)
print("df_ida_fase:", df_ida_fase.shape)

display(df_ian.sort_values(["ano_base", "IAN_faixa_ordem"]).head(12))
display(df_ida_ano.head(10))
display(df_ida_fase.head(10))






OK. Dataframes criados:
df_ian: (9, 5)
df_ida_ano: (3, 8)
df_ida_fase: (25, 5)


,ano_base,IAN_faixa,qtd,pct,IAN_faixa_ordem
0,2022,adequado,259,30.116279,1
1,2022,alerta,573,66.627907,3
2,2022,crítico,28,3.255814,4
3,2023,adequado,462,45.562130,1
4,2023,alerta,538,53.057199,3
5,2023,crítico,14,1.380671,4
6,2024,adequado,622,53.806228,1
7,2024,alerta,531,45.934256,3
8,2024,crítico,3,0.259516,4


,ano_base,n,ida_media,ida_mediana,ida_dp,ida_min,ida_max,delta_ida_media
0,2022,860,6.092907,6.30,2.046209,0.0,9.9,NaN
1,2023,937,6.663394,6.80,1.595277,0.0,10.0,0.570487
2,2024,1055,6.351422,6.75,2.131639,0.0,10.0,-0.311972


,ano_base,Fase_num,n,ida_media,ida_mediana
0,2022,0,190,7.140000,7.30
1,2022,1,192,6.464062,6.80
2,2022,2,155,5.406452,5.50
3,2022,3,148,5.141892,5.20
4,2022,4,76,6.052632,6.10
5,2022,5,60,5.873333,6.55
6,2022,6,18,6.694444,6.85
7,2022,7,21,5.252381,5.00
8,2023,1,173,6.814451,6.90
9,2023,2,199,6.736683,6.80


In [213]:
###############################################################################################################################################
# BLOCO 1 — Criar ~/.netrc com o token do Secret "ITHUB_TOKEN" (sem expor o token)
#
# O que eu faço aqui:
# - Leio o token do Secret do Colab (ITHUB_TOKEN)
# - Se não achar, eu paro com erro claro (pra não gravar "None" no netrc)
# - Gravo ~/.netrc e protejo com permissão 600
###############################################################################################################################################
from google.colab import userdata
import os

token = userdata.get("ITHUB_TOKEN")   # <- do jeito que você definiu
user  = "tivanello"

if not token:
    raise ValueError('Secret "ITHUB_TOKEN" não encontrado ou vazio. Confira em Colab > Secrets.')

path = os.path.expanduser("~/.netrc")
with open(path, "w") as f:
    f.write(f"machine github.com\nlogin {user}\npassword {token}\n")
os.chmod(path, 0o600)

print("OK: ~/.netrc criado/atualizado (token não foi exibido).")


OK: ~/.netrc criado/atualizado (token não foi exibido).


In [214]:
###############################################################################################################################################
# BLOCO 2 — Teste de acesso ao GitHub (sem expor token)
#
# O que eu faço aqui:
# - Chamo a API do GitHub para confirmar:
#   1) qual usuário o token representa
#   2) se o repo tivanello/fase5 é acessível
###############################################################################################################################################
import json, urllib.request
from google.colab import userdata

token = userdata.get("ITHUB_TOKEN")

def gh_get(url):
    req = urllib.request.Request(url)
    req.add_header("Authorization", f"token {token}")
    req.add_header("Accept", "application/vnd.github+json")
    with urllib.request.urlopen(req) as r:
        return json.loads(r.read().decode("utf-8"))

me = gh_get("https://api.github.com/user")
repo = gh_get("https://api.github.com/repos/tivanello/fase5")

print("Usuário autenticado:", me.get("login"))
print("Repo:", repo.get("full_name"))
print("Permissões:", repo.get("permissions"))


Usuário autenticado: tivanello
Repo: tivanello/fase5
Permissões: {'admin': True, 'maintain': True, 'push': True, 'triage': True, 'pull': True}


In [215]:
%%bash
###############################################################################################################################################
# BLOCO A — Clonar repo e entrar em /content/fase5
#
# O que eu faço aqui:
# - Vou para /content
# - Clono o repo se ele não existir
# - Entro no diretório do repo e confirmo caminho
###############################################################################################################################################
cd /content || exit 1
if [ ! -d "/content/fase5/.git" ]; then
  rm -rf /content/fase5
  git clone https://github.com/tivanello/fase5.git
fi
cd /content/fase5 || exit 1
pwd
git remote -v


/content/fase5
origin	https://github.com/tivanello/fase5.git (fetch)
origin	https://github.com/tivanello/fase5.git (push)


In [216]:
###############################################################################################################################################
# Salvar indicadores em data/processed
#
# O que eu faço aqui:
# - Crio a pasta data/processed dentro do repo local (/content/fase5)
# - Exporto os indicadores em CSV e Parquet
# - Mostro os arquivos gerados
###############################################################################################################################################

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_ian.to_csv(os.path.join(out_dir, "df_ian.csv"), index=False, encoding="utf-8")
df_ida_ano.to_csv(os.path.join(out_dir, "df_ida_ano.csv"), index=False, encoding="utf-8")
df_ida_fase.to_csv(os.path.join(out_dir, "df_ida_fase.csv"), index=False, encoding="utf-8")

# Parquet (se der erro, instale: pip install -q pyarrow)
df_ian.to_parquet(os.path.join(out_dir, "df_ian.parquet"), index=False)
df_ida_ano.to_parquet(os.path.join(out_dir, "df_ida_ano.parquet"), index=False)
df_ida_fase.to_parquet(os.path.join(out_dir, "df_ida_fase.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'dummy_update.txt']


In [217]:
%%bash
###############################################################################################################################################
#  Versionar e enviar data/processed para o GitHub
#
# O que eu faço aqui:
# - Adiciono os arquivos de data/processed ao Git
# - Faço commit
# - Faço push para o GitHub
###############################################################################################################################################
cd /content/fase5 || exit 1
git add data/processed
git status
git commit -m "Adiciona indicadores em data/processed" || true
git push origin main


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Everything up-to-date


In [220]:
%%bash
###############################################################################################################################################
# Conferência final: commit no topo + arquivos versionados
###############################################################################################################################################
cd /content/fase5 || exit 1
git log -1 --oneline
ls -la data/processed


fc56ffb Adiciona indicadores em data/processed (automatic update)
total 40
drwxr-xr-x 2 root root 4096 Feb 18 20:21 .
drwxr-xr-x 4 root root 4096 Feb 18 20:14 ..
-rw-r--r-- 1 root root  381 Feb 18 20:47 df_ian.csv
-rw-r--r-- 1 root root 3517 Feb 18 20:47 df_ian.parquet
-rw-r--r-- 1 root root  290 Feb 18 20:47 df_ida_ano.csv
-rw-r--r-- 1 root root 5135 Feb 18 20:47 df_ida_ano.parquet
-rw-r--r-- 1 root root  782 Feb 18 20:47 df_ida_fase.csv
-rw-r--r-- 1 root root 3858 Feb 18 20:47 df_ida_fase.parquet
-rw-r--r-- 1 root root   32 Feb 18 20:21 dummy_update.txt


In [221]:
# ============================================================
# Pergunta 3
# Tema: Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta
#       com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)? -  IEG vs IDA vs IPV (Dispersão + Correlação)
# Objetivo:
#   1) Gerar base para gráfico de dispersão (Power BI):
#      - IEG (eixo X), IDA (eixo Y) e IPV (cor/tamanho)
#   2) Calcular correlações por ano_base e Fase_num:
#      - corr(IEG, IDA), corr(IEG, IPV), corr(IDA, IPV) + n
# Saídas:
#   - df_q3_scatter
#   - df_q3_corr
# ============================================================

import numpy as np
import pandas as pd

# Use seu dataframe principal consolidado aqui:
df = df_fase5.copy()

# Garantir tipos numéricos
for c in ["IEG", "IDA", "IPV"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Garantir ano_base e Fase_num (se existirem)
for c in ["ano_base", "Fase_num"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 1) Base para dispersão (scatter)
cols_scatter = [c for c in ["ano_base", "Fase_num", "IEG", "IDA", "IPV"] if c in df.columns]
df_q3_scatter = df[cols_scatter].dropna(subset=["IEG", "IDA"]).copy()

print("df_q3_scatter -> linhas:", len(df_q3_scatter), "| colunas:", list(df_q3_scatter.columns))

# 2) Correlações por ano_base e Fase_num (se existirem)
group_cols = [c for c in ["ano_base", "Fase_num"] if c in df_q3_scatter.columns]

def corr_safe(a, b):
    a = a.dropna()
    b = b.dropna()
    # precisa de pelo menos 3 pares para fazer sentido
    n = min(len(a), len(b))
    if n < 3:
        return np.nan
    # alinhar índices
    sub = pd.concat([a, b], axis=1).dropna()
    if len(sub) < 3:
        return np.nan
    return sub.iloc[:, 0].corr(sub.iloc[:, 1])

if group_cols:
    rows = []
    for keys, g in df_q3_scatter.groupby(group_cols, dropna=False):
        d = dict(zip(group_cols, keys if isinstance(keys, tuple) else (keys,)))
        d["corr_IEG_IDA"] = corr_safe(g["IEG"], g["IDA"])
        d["corr_IEG_IPV"] = corr_safe(g["IEG"], g["IPV"]) if "IPV" in g.columns else np.nan
        d["corr_IDA_IPV"] = corr_safe(g["IDA"], g["IPV"]) if "IPV" in g.columns else np.nan
        d["n"] = len(g)
        rows.append(d)

    df_q3_corr = pd.DataFrame(rows).sort_values(group_cols)
    print("\ndf_q3_corr (amostra):")
    print(df_q3_corr.head(10))
else:
    df_q3_corr = pd.DataFrame()
    print("\nSem ano_base/Fase_num -> correlação por grupo não foi gerada.")


df_q3_scatter -> linhas: 2852 | colunas: ['ano_base', 'Fase_num', 'IEG', 'IDA', 'IPV']

df_q3_corr (amostra):
   ano_base Fase_num  corr_IEG_IDA  corr_IEG_IPV  corr_IDA_IPV    n
0      2022        0      0.450592      0.431056      0.487454  190
1      2022        1      0.553685      0.444143      0.651949  192
2      2022        2      0.394289      0.423781      0.544388  155
3      2022        3      0.686982      0.776172      0.653425  148
4      2022        4      0.689837      0.676765      0.760824   76
5      2022        5      0.741022      0.670497      0.672272   60
6      2022        6      0.801540      0.851491      0.671248   18
7      2022        7      0.693013      0.557890      0.373152   21
8      2023        1      0.411002      0.396233      0.497385  173
9      2023        2      0.397772      0.455233      0.483041  199


In [ ]:
# ============================================================
# Pergunta 3 - Exportar (CSV + Parquet) os dataframes gerados:
#   - df_q3_scatter = dados ponto a ponto (para o gráfico)
#   - df_q3_corr = “placar” por ano/fase (para card/tabela e interpretação)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q3_scatter.to_csv(os.path.join(out_dir, "df_q3_scatter.csv"), index=False, encoding="utf-8")
df_q3_corr.to_csv(os.path.join(out_dir, "df_q3_corr.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q3_scatter.to_parquet(os.path.join(out_dir, "df_q3_scatter.parquet"), index=False)
df_q3_corr.to_parquet(os.path.join(out_dir, "df_q3_corr.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


In [222]:
# ============================================================
# Pergunta 4
# Tema: Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA)
#       são coerentes com seu desempenho real (IDA) e engajamento (IEG)?
#       Autoavaliação (IAA) é coerente com desempenho (IDA) e engajamento (IEG)?
# Objetivo:
#   1) Base para dispersão (Power BI):
#      - IAA (eixo X) vs IDA (eixo Y), com IEG como cor/tamanho (tooltip)
#   2) Quadrantes de coerência (por mediana):
#      - IAA alto/baixo x IDA alto/baixo (contagem por ano_base e fase)
# Saídas:
#   - df_q4_scatter
#   - df_q4_quadrantes
# ============================================================

import numpy as np
import pandas as pd

df = df_fase5.copy()

# Tipos numéricos
for c in ["IAA", "IDA", "IEG"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

for c in ["ano_base", "Fase_num"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 1) Base para scatter: IAA x IDA, com IEG junto
cols_scatter = [c for c in ["ano_base", "Fase_num", "IAA", "IDA", "IEG"] if c in df.columns]
df_q4_scatter = df[cols_scatter].dropna(subset=["IAA", "IDA"]).copy()

print("df_q4_scatter -> linhas:", len(df_q4_scatter), "| colunas:", list(df_q4_scatter.columns))

# 2) Quadrantes por mediana (robusto e fácil de explicar)
tmp = df_q4_scatter.dropna(subset=["IAA", "IDA"]).copy()
iaa_med = tmp["IAA"].median()
ida_med = tmp["IDA"].median()

tmp["IAA_nivel"] = np.where(tmp["IAA"] >= iaa_med, "IAA_alto", "IAA_baixo")
tmp["IDA_nivel"] = np.where(tmp["IDA"] >= ida_med, "IDA_alto", "IDA_baixo")
tmp["quadrante"] = tmp["IAA_nivel"] + " x " + tmp["IDA_nivel"]

group_cols = [c for c in ["ano_base", "Fase_num", "quadrante"] if c in tmp.columns]
df_q4_quadrantes = (
    tmp.groupby(group_cols)
       .size()
       .reset_index(name="qtd")
       .sort_values(group_cols)
)

print("\ndf_q4_quadrantes (amostra):")
print(df_q4_quadrantes.head(12))


df_q4_scatter -> linhas: 2851 | colunas: ['ano_base', 'Fase_num', 'IAA', 'IDA', 'IEG']

df_q4_quadrantes (amostra):
    ano_base  Fase_num              quadrante  qtd
0       2022         0    IAA_alto x IDA_alto   94
1       2022         0   IAA_alto x IDA_baixo   44
2       2022         0   IAA_baixo x IDA_alto   32
3       2022         0  IAA_baixo x IDA_baixo   20
4       2022         1    IAA_alto x IDA_alto   61
5       2022         1   IAA_alto x IDA_baixo   46
6       2022         1   IAA_baixo x IDA_alto   44
7       2022         1  IAA_baixo x IDA_baixo   41
8       2022         2    IAA_alto x IDA_alto   23
9       2022         2   IAA_alto x IDA_baixo   63
10      2022         2   IAA_baixo x IDA_alto   24
11      2022         2  IAA_baixo x IDA_baixo   45


In [223]:
# ============================================================
# Pergunta 4 - Exportar (CSV + Parquet) os dataframes gerados:
#   - df_q4_scatter = base ponto a ponto (IAA x IDA, com IEG junto)
#   - df_q4_quadrantes = “placar” de coerência (quadrantes por ano/fase)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q4_scatter.to_csv(os.path.join(out_dir, "df_q4_scatter.csv"), index=False, encoding="utf-8")
df_q4_quadrantes.to_csv(os.path.join(out_dir, "df_q4_quadrantes.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q4_scatter.to_parquet(os.path.join(out_dir, "df_q4_scatter.parquet"), index=False)
df_q4_quadrantes.to_parquet(os.path.join(out_dir, "df_q4_quadrantes.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'dummy_update.txt']


In [226]:
# ============================================================
# Pergunta 5
# Tema: Há padrões psicossociais (IPS) que
#       antecedem quedas de desempenho acadêmico ou de engajamento?
#       IPS antecede queda de desempenho?
# Chave do aluno:
#   - ra_num como ID principal (numérico e estável)
# Objetivo:
#   1) Calcular desempenho do ano anterior (INDE_prev) e IPS do ano anterior (IPS_prev)
#   2) Marcar queda quando delta_INDE < 0
# Saídas:
#   - df_q5_flags  (linha a linha: queda, INDE_prev, IPS_prev)
#   - df_q5_resumo (comparação IPS_prev: caiu vs não caiu, por ano_base e fase)
# ============================================================

import numpy as np
import pandas as pd

df = df_fase5.copy()

COL_ID = "ra_num"   # ID do aluno (numérico)
COL_ID_TXT = "RA"   # opcional: para exibição

# Conferência rápida das colunas necessárias
needed = ["ano_base", "Fase_num", "IPS", "INDE_final", COL_ID]
missing = [c for c in needed if c not in df.columns]
if missing:
    print("ERRO: faltam colunas para Pergunta 5:", missing)
else:
    # Tipos
    for c in ["ano_base", "Fase_num", "IPS", "INDE_final", COL_ID]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Ordenar por aluno e ano
    df = df.sort_values([COL_ID, "ano_base"]).copy()

    # Ano anterior (por aluno)
    df["INDE_prev"] = df.groupby(COL_ID)["INDE_final"].shift(1)
    df["IPS_prev"]  = df.groupby(COL_ID)["IPS"].shift(1)

    # Variação de desempenho
    df["delta_INDE"] = df["INDE_final"] - df["INDE_prev"]

    # Queda (regra simples)
    df["queda"] = np.where(df["delta_INDE"] < 0, 1, 0)

    # Base linha a linha
    cols_flags = [COL_ID, "ano_base", "Fase_num", "INDE_final", "INDE_prev", "delta_INDE", "queda", "IPS", "IPS_prev"]
    if COL_ID_TXT in df.columns:
        cols_flags = [COL_ID_TXT] + cols_flags

    df_q5_flags = df[cols_flags].copy()

    print("df_q5_flags -> linhas:", len(df_q5_flags), "| colunas:", list(df_q5_flags.columns))

    # Resumo: compara IPS_prev entre quem caiu vs não caiu
    df_q5_resumo = (
        df_q5_flags.dropna(subset=["IPS_prev", "delta_INDE"])
        .groupby(["ano_base", "Fase_num", "queda"], dropna=False)
        .agg(
            n=("delta_INDE", "size"),
            ips_prev_media=("IPS_prev", "mean"),
            delta_inde_medio=("delta_INDE", "mean"),
        )
        .reset_index()
        .sort_values(["ano_base", "Fase_num", "queda"])
    )

    print("\ndf_q5_resumo (amostra):")
    print(df_q5_resumo.head(12))


df_q5_flags -> linhas: 3030 | colunas: ['RA', 'ra_num', 'ano_base', 'Fase_num', 'INDE_final', 'INDE_prev', 'delta_INDE', 'queda', 'IPS', 'IPS_prev']

df_q5_resumo (amostra):
    ano_base  Fase_num  queda   n  ips_prev_media  delta_inde_medio
0       2023         1      0  69        7.049275          0.531522
1       2023         1      1  47        6.953191         -0.541620
2       2023         2      0  72        6.894444          0.598373
3       2023         2      1  57        7.094737         -0.592820
4       2023         3      0  47        6.702128          0.467446
5       2023         3      1  56        6.966071         -0.543760
6       2023         4      0  29        6.451724          0.577007
7       2023         4      1  37        6.556757         -0.551560
8       2023         5      0   8        6.250000          0.518200
9       2023         5      1  39        6.510256         -0.704133
10      2023         6      0  12        6.658333          0.713486
11      20

In [227]:
# ============================================================
# Pergunta 5 - Exportar (CSV + Parquet) os dataframes gerados:
#   - df_q5_flags  = base ponto a ponto (por aluno/ano: queda, INDE_prev, IPS_prev)
#   - df_q5_resumo = “placar” (IPS_prev: caiu vs não caiu, por ano_base e fase)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q5_flags.to_csv(os.path.join(out_dir, "df_q5_flags.csv"), index=False, encoding="utf-8")
df_q5_resumo.to_csv(os.path.join(out_dir, "df_q5_resumo.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q5_flags.to_parquet(os.path.join(out_dir, "df_q5_flags.parquet"), index=False)
df_q5_resumo.to_parquet(os.path.join(out_dir, "df_q5_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'dummy_update.txt']


In [228]:
# ============================================================
# Pergunta 6
# Tema:  As avaliações psicopedagógicas
#        (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?
#        IPP confirma ou contradiz IAN?
# Ideia:
#   1) Base ponto a ponto: IAN x IPP
#   2) Quadrantes por mediana:
#      - IAN_alto/baixo x IPP_alto/baixo
# Saídas:
#   - df_q6_base
#   - df_q6_quadrantes
# ============================================================

import numpy as np
import pandas as pd

df = df_fase5.copy()

COL_ID = "ra_num"   # ID principal
COL_ID_TXT = "RA"   # opcional para exibição

# Checagem de colunas necessárias
needed = ["ano_base", "Fase_num", "IAN", "IPP", COL_ID]
missing = [c for c in needed if c not in df.columns]
if missing:
    print("ERRO: faltam colunas para Pergunta 6:", missing)
else:
    # Tipos
    for c in ["ano_base", "Fase_num", "IAN", "IPP", COL_ID]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Base ponto a ponto
    cols = [COL_ID, "ano_base", "Fase_num", "IAN", "IPP"]
    if COL_ID_TXT in df.columns:
        cols = [COL_ID_TXT] + cols

    df_q6_base = df[cols].dropna(subset=["IAN", "IPP"]).copy()

    print("df_q6_base -> linhas:", len(df_q6_base), "| colunas:", list(df_q6_base.columns))

    # Quadrantes por mediana
    ian_med = df_q6_base["IAN"].median()
    ipp_med = df_q6_base["IPP"].median()

    tmp = df_q6_base.copy()
    tmp["IAN_nivel"] = np.where(tmp["IAN"] >= ian_med, "IAN_alto", "IAN_baixo")
    tmp["IPP_nivel"] = np.where(tmp["IPP"] >= ipp_med, "IPP_alto", "IPP_baixo")
    tmp["quadrante"] = tmp["IAN_nivel"] + " x " + tmp["IPP_nivel"]

    df_q6_quadrantes = (
        tmp.groupby(["ano_base", "Fase_num", "quadrante"], dropna=False)
           .size()
           .reset_index(name="qtd")
           .sort_values(["ano_base", "Fase_num", "quadrante"])
    )

    print("\ndf_q6_quadrantes (amostra):")
    print(df_q6_quadrantes.head(12))


df_q6_base -> linhas: 1992 | colunas: ['RA', 'ra_num', 'ano_base', 'Fase_num', 'IAN', 'IPP']

df_q6_quadrantes (amostra):
    ano_base  Fase_num              quadrante  qtd
0       2023         1    IAN_alto x IPP_alto  122
1       2023         1   IAN_alto x IPP_baixo   50
2       2023         1  IAN_baixo x IPP_baixo    1
3       2023         2    IAN_alto x IPP_alto  161
4       2023         2   IAN_alto x IPP_baixo   37
5       2023         2   IAN_baixo x IPP_alto    1
6       2023         2  IAN_baixo x IPP_baixo    1
7       2023         3    IAN_alto x IPP_alto   86
8       2023         3   IAN_alto x IPP_baixo   44
9       2023         3   IAN_baixo x IPP_alto    2
10      2023         4    IAN_alto x IPP_alto   65
11      2023         4   IAN_alto x IPP_baixo   24


In [229]:
# ============================================================
# Pergunta 6 - Exportar (CSV + Parquet) os dataframes gerados:
#   - df_q6_base       = base ponto a ponto (IAN x IPP, por aluno/ano)
#   - df_q6_quadrantes = “placar” (quadrantes IAN alto/baixo x IPP alto/baixo)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q6_base.to_csv(os.path.join(out_dir, "df_q6_base.csv"), index=False, encoding="utf-8")
df_q6_quadrantes.to_csv(os.path.join(out_dir, "df_q6_quadrantes.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q6_base.to_parquet(os.path.join(out_dir, "df_q6_base.parquet"), index=False)
df_q6_quadrantes.to_parquet(os.path.join(out_dir, "df_q6_quadrantes.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'dummy_update.txt']


In [230]:
# ============================================================
# Pergunta 7
# Tema: Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?
#       O que mais influencia IPV? (ranking de fatores)
# Método (baseline simples e explicável):
#   - calcular correlação de IPV com cada indicador disponível
#   - ordenar por |correlação| (força da associação)
# Saída:
#   - df_q7_rank_ipv
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

# Checagem
if "IPV" not in df.columns:
    print("ERRO: não existe a coluna IPV no dataframe.")
else:
    df["IPV"] = pd.to_numeric(df["IPV"], errors="coerce")

    # Lista de candidatos (ajuste se quiser)
    candidatos = ["IDA", "IEG", "IAA", "IPS", "IPP", "IAN", "INDE_final"]
    candidatos = [c for c in candidatos if c in df.columns]

    rows = []
    for c in candidatos:
        x = pd.to_numeric(df[c], errors="coerce")
        sub = pd.concat([x, df["IPV"]], axis=1).dropna()
        if len(sub) < 30:   # mínimo pra não virar estatística de guardanapo
            continue
        corr = sub.iloc[:, 0].corr(sub.iloc[:, 1])
        rows.append({"variavel": c, "corr": corr, "abs_corr": abs(corr), "n": len(sub)})

    df_q7_rank_ipv = pd.DataFrame(rows).sort_values(["abs_corr", "n"], ascending=[False, False])

    print("df_q7_rank_ipv:")
    print(df_q7_rank_ipv)


df_q7_rank_ipv:
     variavel      corr  abs_corr     n
6  INDE_final  0.720896  0.720896  2845
4         IPP  0.606649  0.606649  1992
1         IEG  0.558200  0.558200  2852
0         IDA  0.557072  0.557072  2851
5         IAN  0.148880  0.148880  2852
2         IAA  0.062728  0.062728  2852
3         IPS -0.048750  0.048750  2846


In [231]:
# ============================================================
# Pergunta 7 - Exportar (CSV + Parquet) o dataframe gerado:
#   - df_q7_rank_ipv = ranking de fatores associados ao IPV (corr e |corr|)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q7_rank_ipv.to_csv(os.path.join(out_dir, "df_q7_rank_ipv.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q7_rank_ipv.to_parquet(os.path.join(out_dir, "df_q7_rank_ipv.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'dummy_update.txt']


In [232]:
# ============================================================
# Pergunta 8
# Tema: Quais combinações de indicadores (IDA + IEG + IPS + IPP) elevam mais a nota global do aluno (INDE)?
#       Quais combinações elevam o INDE? (heatmap/segmentos)
# Método:
#   1) Criar faixas (quartis) para IDA, IEG, IPS, IPP
#   2) Agrupar pelas faixas e calcular:
#      - média do INDE_final
#      - quantidade (n)
# Saída:
#   - df_q8_heatmap
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

# Colunas necessárias
needed = ["INDE_final", "IDA", "IEG", "IPS", "IPP", "ano_base", "Fase_num"]
missing = [c for c in needed if c not in df.columns]
if missing:
    print("ERRO: faltam colunas para Pergunta 8:", missing)
else:
    # Tipos numéricos
    for c in ["INDE_final", "IDA", "IEG", "IPS", "IPP", "ano_base", "Fase_num"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Remove linhas sem os indicadores principais
    tmp = df[["ano_base", "Fase_num", "INDE_final", "IDA", "IEG", "IPS", "IPP"]].dropna().copy()

    # Faixas por quartis (Q1..Q4)
    tmp["IDA_faixa"] = pd.qcut(tmp["IDA"], 4, labels=["Q1_baixo", "Q2", "Q3", "Q4_alto"])
    tmp["IEG_faixa"] = pd.qcut(tmp["IEG"], 4, labels=["Q1_baixo", "Q2", "Q3", "Q4_alto"])
    tmp["IPS_faixa"] = pd.qcut(tmp["IPS"], 4, labels=["Q1_baixo", "Q2", "Q3", "Q4_alto"])
    tmp["IPP_faixa"] = pd.qcut(tmp["IPP"], 4, labels=["Q1_baixo", "Q2", "Q3", "Q4_alto"])

    # Agrupar por ano/fase e pelas faixas
    df_q8_heatmap = (
        tmp.groupby(["ano_base", "Fase_num", "IDA_faixa", "IEG_faixa", "IPS_faixa", "IPP_faixa"], dropna=False)
           .agg(
               n=("INDE_final", "size"),
               inde_medio=("INDE_final", "mean")
           )
           .reset_index()
           .sort_values(["ano_base", "Fase_num", "IDA_faixa", "IEG_faixa", "IPS_faixa", "IPP_faixa"])
    )

    print("df_q8_heatmap -> linhas:", len(df_q8_heatmap))
    print(df_q8_heatmap.head(12))


df_q8_heatmap -> linhas: 3584
    ano_base  Fase_num IDA_faixa IEG_faixa IPS_faixa IPP_faixa  n  inde_medio
0       2023         1  Q1_baixo  Q1_baixo  Q1_baixo  Q1_baixo  0         NaN
1       2023         1  Q1_baixo  Q1_baixo  Q1_baixo        Q2  1    5.331167
2       2023         1  Q1_baixo  Q1_baixo  Q1_baixo        Q3  0         NaN
3       2023         1  Q1_baixo  Q1_baixo  Q1_baixo   Q4_alto  0         NaN
4       2023         1  Q1_baixo  Q1_baixo        Q2  Q1_baixo  1    6.138367
5       2023         1  Q1_baixo  Q1_baixo        Q2        Q2  0         NaN
6       2023         1  Q1_baixo  Q1_baixo        Q2        Q3  0         NaN
7       2023         1  Q1_baixo  Q1_baixo        Q2   Q4_alto  0         NaN
8       2023         1  Q1_baixo  Q1_baixo        Q3  Q1_baixo  0         NaN
9       2023         1  Q1_baixo  Q1_baixo        Q3        Q2  0         NaN
10      2023         1  Q1_baixo  Q1_baixo        Q3        Q3  0         NaN
11      2023         1  Q1_baixo  

/tmp/ipython-input-2012999107.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tmp.groupby(["ano_base", "Fase_num", "IDA_faixa", "IEG_faixa", "IPS_faixa", "IPP_faixa"], dropna=False)


In [233]:
# ============================================================
# Pergunta 8 - Exportar (CSV + Parquet) o dataframe gerado:
#   - df_q8_heatmap = combinações (faixas) que elevam o INDE (média e n)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV
df_q8_heatmap.to_csv(os.path.join(out_dir, "df_q8_heatmap.csv"), index=False, encoding="utf-8")

# Parquet (se der erro: !pip install -q pyarrow)
df_q8_heatmap.to_parquet(os.path.join(out_dir, "df_q8_heatmap.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_heatmap.parquet', 'dummy_update.txt']


In [234]:
# ============================================================
# Pergunta 9 (NOVA)
# Tema: "Quem priorizar?" (segmentos de risco)
# Ideia:
#   Criar um score simples e explicável usando indicadores disponíveis:
#     - IAN alto  -> aumenta risco
#     - INDE_final baixo -> aumenta risco
#     - IPS baixo -> aumenta risco
#     - IEG baixo -> aumenta risco
#     - (opcional) queda recente (delta_INDE < 0) -> aumenta risco
# Saídas:
#   - df_q9_segmentos (aluno a aluno com score e categoria)
#   - df_q9_resumo (contagem por categoria, por ano_base e fase)
# ============================================================

import numpy as np
import pandas as pd

df = df_fase5.copy()

COL_ID = "ra_num"
COL_ID_TXT = "RA"

needed = ["ano_base", "Fase_num", COL_ID, "IAN", "INDE_final", "IPS", "IEG"]
missing = [c for c in needed if c not in df.columns]
if missing:
    print("ERRO: faltam colunas para Pergunta 9:", missing)
else:
    # tipos
    for c in ["ano_base", "Fase_num", COL_ID, "IAN", "INDE_final", "IPS", "IEG"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # base mínima
    cols = [COL_ID, "ano_base", "Fase_num", "IAN", "INDE_final", "IPS", "IEG"]
    if COL_ID_TXT in df.columns:
        cols = [COL_ID_TXT] + cols
    tmp = df[cols].dropna(subset=["IAN", "INDE_final", "IPS", "IEG"]).copy()

    # (opcional) queda recente usando INDE_final do ano anterior
    tmp = tmp.sort_values([COL_ID, "ano_base"])
    tmp["INDE_prev"] = tmp.groupby(COL_ID)["INDE_final"].shift(1)
    tmp["delta_INDE"] = tmp["INDE_final"] - tmp["INDE_prev"]
    tmp["queda_recente"] = np.where(tmp["delta_INDE"] < 0, 1, 0)

    # normalização simples por percentil (0 a 1) para combinar sem briga de escala
    def pct_rank(s):
        return s.rank(pct=True)

    # risco sobe com: IAN alto, INDE baixo, IPS baixo, IEG baixo, queda_recente
    tmp["r_ian"] = pct_rank(tmp["IAN"])                 # alto = pior
    tmp["r_inde"] = 1 - pct_rank(tmp["INDE_final"])     # baixo = pior
    tmp["r_ips"] = 1 - pct_rank(tmp["IPS"])             # baixo = pior
    tmp["r_ieg"] = 1 - pct_rank(tmp["IEG"])             # baixo = pior

    # pesos (simples e explicáveis)
    w_ian, w_inde, w_ips, w_ieg, w_queda = 0.30, 0.30, 0.20, 0.15, 0.05

    tmp["score_risco"] = (
        w_ian  * tmp["r_ian"] +
        w_inde * tmp["r_inde"] +
        w_ips  * tmp["r_ips"] +
        w_ieg  * tmp["r_ieg"] +
        w_queda * tmp["queda_recente"]
    )

    # categorias por tercis (top 1/3 = alto risco)
    p66 = tmp["score_risco"].quantile(0.66)
    p33 = tmp["score_risco"].quantile(0.33)

    tmp["categoria_risco"] = np.where(
        tmp["score_risco"] >= p66, "ALTO",
        np.where(tmp["score_risco"] <= p33, "BAIXO", "MEDIO")
    )

    # saída aluno a aluno
    keep = [COL_ID, "ano_base", "Fase_num", "categoria_risco", "score_risco",
            "IAN", "INDE_final", "IPS", "IEG", "queda_recente", "delta_INDE"]
    if COL_ID_TXT in tmp.columns:
        keep = [COL_ID_TXT] + keep

    df_q9_segmentos = tmp[keep].sort_values(["ano_base", "Fase_num", "categoria_risco", "score_risco"], ascending=[True, True, True, False]).copy()

    # resumo por ano/fase
    df_q9_resumo = (
        df_q9_segmentos.groupby(["ano_base", "Fase_num", "categoria_risco"], dropna=False)
        .size()
        .reset_index(name="qtd")
        .sort_values(["ano_base", "Fase_num", "categoria_risco"])
    )

    print("df_q9_segmentos -> linhas:", len(df_q9_segmentos))
    print(df_q9_segmentos.head(10))
    print("\ndf_q9_resumo (amostra):")
    print(df_q9_resumo.head(12))


df_q9_segmentos -> linhas: 2845
         RA  ra_num  ano_base  Fase_num categoria_risco  score_risco   IAN  \
851  RA-852     852      2022         0            ALTO     0.802091  10.0   
694  RA-695     695      2022         0            ALTO     0.772583  10.0   
723  RA-724     724      2022         0            ALTO     0.718533  10.0   
738  RA-739     739      2022         0            ALTO     0.710835  10.0   
823  RA-824     824      2022         0            ALTO     0.703383  10.0   
775  RA-776     776      2022         0            ALTO     0.702100   5.0   
799  RA-800     800      2022         0            ALTO     0.676019  10.0   
774  RA-775     775      2022         0            ALTO     0.670097   5.0   
711  RA-712     712      2022         0            ALTO     0.656942   5.0   
812  RA-813     813      2022         0            ALTO     0.655589  10.0   

     INDE_final  IPS  IEG  queda_recente  delta_INDE  
851       5.666  5.0  6.8              0         NaN  

In [235]:
# ============================================================
# Pergunta 9 - Exportar (CSV + Parquet) os dataframes gerados:
#   - df_q9_segmentos = alunos com score e categoria de risco
#   - df_q9_resumo    = contagem por risco (ano_base e fase)
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q9_segmentos.to_csv(os.path.join(out_dir, "df_q9_segmentos.csv"), index=False, encoding="utf-8")
df_q9_resumo.to_csv(os.path.join(out_dir, "df_q9_resumo.csv"), index=False, encoding="utf-8")

df_q9_segmentos.to_parquet(os.path.join(out_dir, "df_q9_segmentos.parquet"), index=False)
df_q9_resumo.to_parquet(os.path.join(out_dir, "df_q9_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_heatmap.parquet', 'df_q9_resumo.csv', 'df_q9_resumo.parquet', 'df_q9_segmentos.csv', 'df_q9_segmentos.parquet', 'dummy_update.txt']


In [236]:
# ============================================================
# Pergunta 11-A — Insights e criatividade
# Tema: Perfis de intervenção (sem “score”), por regras simples e explicáveis
# Ideia:
#   Criar perfis operacionais (para ação) usando indicadores já existentes.
# Perfis:
#   1) CRITICO_PEDAGOGICO   -> INDE_final baixo OU IDA baixo
#   2) CRITICO_ENGAJAMENTO  -> IEG baixo
#   3) CRITICO_SOCIOEMO     -> IPS baixo
#   4) ALERTA_DOCENTE       -> IPP alto com INDE_final ainda ok (prevenção)
# Regras:
#   - cortes por quartis (Q1/Q4) para ser robusto e simples de explicar
# Saídas:
#   - df_q11A_perfis_alunos  (aluno a aluno com perfil)
#   - df_q11A_perfis_resumo  (contagem por perfil, ano_base e fase)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

COL_ID = "ra_num"
COL_TXT = "RA"

needed = ["ano_base", "Fase_num", COL_ID, "INDE_final", "IDA", "IEG", "IPS", "IPP"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-A:", missing)
else:
    # Tipos
    for c in ["ano_base", "Fase_num", COL_ID, "INDE_final", "IDA", "IEG", "IPS", "IPP"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    cols = [COL_ID, "ano_base", "Fase_num", "INDE_final", "IDA", "IEG", "IPS", "IPP"]
    if COL_TXT in df.columns:
        cols = [COL_TXT] + cols

    tmp = df[cols].dropna(subset=["INDE_final", "IDA", "IEG", "IPS", "IPP"]).copy()

    # Quartis (globais)
    q1_inde = tmp["INDE_final"].quantile(0.25)
    q1_ida  = tmp["IDA"].quantile(0.25)
    q1_ieg  = tmp["IEG"].quantile(0.25)
    q1_ips  = tmp["IPS"].quantile(0.25)
    q4_ipp  = tmp["IPP"].quantile(0.75)

    # Flags
    tmp["f_pedagogico"]  = ((tmp["INDE_final"] <= q1_inde) | (tmp["IDA"] <= q1_ida)).astype(int)
    tmp["f_engajamento"] = (tmp["IEG"] <= q1_ieg).astype(int)
    tmp["f_socioemo"]    = (tmp["IPS"] <= q1_ips).astype(int)

    # Alerta docente: IPP alto (Q4) mas INDE_final não está no fundo (acima de Q1)
    tmp["f_alerta_docente"] = ((tmp["IPP"] >= q4_ipp) & (tmp["INDE_final"] > q1_inde)).astype(int)

    # Perfil principal (prioridade simples)
    # Se tiver mais de um flag, segue a prioridade:
    # socioemo > pedagogico > engajamento > alerta_docente > ok
    def define_perfil(row):
        if row["f_socioemo"] == 1:
            return "CRITICO_SOCIOEMO"
        if row["f_pedagogico"] == 1:
            return "CRITICO_PEDAGOGICO"
        if row["f_engajamento"] == 1:
            return "CRITICO_ENGAJAMENTO"
        if row["f_alerta_docente"] == 1:
            return "ALERTA_DOCENTE"
        return "OK"

    tmp["perfil_intervencao"] = tmp.apply(define_perfil, axis=1)

    # Saídas
    keep = [COL_ID, "ano_base", "Fase_num", "perfil_intervencao",
            "INDE_final", "IDA", "IEG", "IPS", "IPP",
            "f_pedagogico", "f_engajamento", "f_socioemo", "f_alerta_docente"]
    if COL_TXT in tmp.columns:
        keep = [COL_TXT] + keep

    df_q11A_perfis_alunos = tmp[keep].copy()

    df_q11A_perfis_resumo = (
        df_q11A_perfis_alunos
        .groupby(["ano_base", "Fase_num", "perfil_intervencao"], dropna=False)
        .size()
        .reset_index(name="qtd")
        .sort_values(["ano_base", "Fase_num", "perfil_intervencao"])
    )

    print("df_q11A_perfis_alunos -> linhas:", len(df_q11A_perfis_alunos))
    print("df_q11A_perfis_resumo (amostra):")
    print(df_q11A_perfis_resumo.head(12))


df_q11A_perfis_alunos -> linhas: 1985
df_q11A_perfis_resumo (amostra):
    ano_base  Fase_num   perfil_intervencao  qtd
0       2023         1       ALERTA_DOCENTE   48
1       2023         1  CRITICO_ENGAJAMENTO    7
2       2023         1   CRITICO_PEDAGOGICO   23
3       2023         1     CRITICO_SOCIOEMO   47
4       2023         1                   OK   48
5       2023         2       ALERTA_DOCENTE   26
6       2023         2  CRITICO_ENGAJAMENTO    4
7       2023         2   CRITICO_PEDAGOGICO   23
8       2023         2     CRITICO_SOCIOEMO  131
9       2023         2                   OK   15
10      2023         3       ALERTA_DOCENTE    7
11      2023         3  CRITICO_ENGAJAMENTO    6


In [237]:
# ============================================================
# Pergunta 11-A — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11A_perfis_alunos
#   - df_q11A_perfis_resumo
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11A_perfis_alunos.to_csv(os.path.join(out_dir, "df_q11A_perfis_alunos.csv"), index=False, encoding="utf-8")
df_q11A_perfis_resumo.to_csv(os.path.join(out_dir, "df_q11A_perfis_resumo.csv"), index=False, encoding="utf-8")

df_q11A_perfis_alunos.to_parquet(os.path.join(out_dir, "df_q11A_perfis_alunos.parquet"), index=False)
df_q11A_perfis_resumo.to_parquet(os.path.join(out_dir, "df_q11A_perfis_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_heatmap.parquet', 'df_q9_resumo.csv', 'df_q9_resumo.parquet', 'df_q9_segmentos.csv', 'df_q9_segmentos.parquet', 'dummy_update.txt']


In [238]:
# ============================================================
# Pergunta 11-B — Insights e criatividade
# Tema: "Gap de percepção" + "Risco silencioso"
# Ideias:
#   1) Gap de percepção (IAA vs IDA):
#      - superestima: IAA alto e IDA baixo
#      - subestima : IAA baixo e IDA alto
#   2) Risco silencioso:
#      - IEG alto e IDA baixo (engajado, mas não aprende como esperado)
# Critérios:
#   - cortes por mediana (simples e explicável)
# Saídas:
#   - df_q11B_flags_alunos  (aluno a aluno com flags e tipo)
#   - df_q11B_resumo        (contagens por tipo, ano_base e fase)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

COL_ID = "ra_num"
COL_TXT = "RA"

needed = ["ano_base", "Fase_num", COL_ID, "IAA", "IDA", "IEG"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-B:", missing)
else:
    for c in ["ano_base", "Fase_num", COL_ID, "IAA", "IDA", "IEG"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    cols = [COL_ID, "ano_base", "Fase_num", "IAA", "IDA", "IEG"]
    if COL_TXT in df.columns:
        cols = [COL_TXT] + cols

    tmp = df[cols].dropna(subset=["IAA", "IDA", "IEG"]).copy()

    # Cortes (medianas)
    iaa_med = tmp["IAA"].median()
    ida_med = tmp["IDA"].median()
    ieg_med = tmp["IEG"].median()

    tmp["IAA_nivel"] = np.where(tmp["IAA"] >= iaa_med, "IAA_alto", "IAA_baixo")
    tmp["IDA_nivel"] = np.where(tmp["IDA"] >= ida_med, "IDA_alto", "IDA_baixo")
    tmp["IEG_nivel"] = np.where(tmp["IEG"] >= ieg_med, "IEG_alto", "IEG_baixo")

    # Gap de percepção
    tmp["superestima"] = ((tmp["IAA_nivel"] == "IAA_alto") & (tmp["IDA_nivel"] == "IDA_baixo")).astype(int)
    tmp["subestima"]   = ((tmp["IAA_nivel"] == "IAA_baixo") & (tmp["IDA_nivel"] == "IDA_alto")).astype(int)

    # Risco silencioso
    tmp["risco_silencioso"] = ((tmp["IEG_nivel"] == "IEG_alto") & (tmp["IDA_nivel"] == "IDA_baixo")).astype(int)

    # Tipo principal (um rótulo por linha)
    def define_tipo(row):
        if row["risco_silencioso"] == 1:
            return "RISCO_SILENCIOSO"
        if row["superestima"] == 1:
            return "GAP_SUPERESTIMA"
        if row["subestima"] == 1:
            return "GAP_SUBESTIMA"
        return "SEM_ALERTA"

    tmp["tipo_alerta"] = tmp.apply(define_tipo, axis=1)

    keep = [COL_ID, "ano_base", "Fase_num", "tipo_alerta",
            "IAA", "IDA", "IEG",
            "IAA_nivel", "IDA_nivel", "IEG_nivel",
            "superestima", "subestima", "risco_silencioso"]
    if COL_TXT in tmp.columns:
        keep = [COL_TXT] + keep

    df_q11B_flags_alunos = tmp[keep].copy()

    df_q11B_resumo = (
        df_q11B_flags_alunos
        .groupby(["ano_base", "Fase_num", "tipo_alerta"], dropna=False)
        .size()
        .reset_index(name="qtd")
        .sort_values(["ano_base", "Fase_num", "tipo_alerta"])
    )

    print("df_q11B_flags_alunos -> linhas:", len(df_q11B_flags_alunos))
    print("df_q11B_resumo (amostra):")
    print(df_q11B_resumo.head(12))


df_q11B_flags_alunos -> linhas: 2851
df_q11B_resumo (amostra):
    ano_base  Fase_num       tipo_alerta  qtd
0       2022         0     GAP_SUBESTIMA   32
1       2022         0   GAP_SUPERESTIMA   34
2       2022         0  RISCO_SILENCIOSO   14
3       2022         0        SEM_ALERTA  110
4       2022         1     GAP_SUBESTIMA   44
5       2022         1   GAP_SUPERESTIMA   22
6       2022         1  RISCO_SILENCIOSO   41
7       2022         1        SEM_ALERTA   85
8       2022         2     GAP_SUBESTIMA   24
9       2022         2   GAP_SUPERESTIMA   41
10      2022         2  RISCO_SILENCIOSO   38
11      2022         2        SEM_ALERTA   52


In [239]:
# ============================================================
# Pergunta 11-B — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11B_flags_alunos
#   - df_q11B_resumo
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11B_flags_alunos.to_csv(os.path.join(out_dir, "df_q11B_flags_alunos.csv"), index=False, encoding="utf-8")
df_q11B_resumo.to_csv(os.path.join(out_dir, "df_q11B_resumo.csv"), index=False, encoding="utf-8")

df_q11B_flags_alunos.to_parquet(os.path.join(out_dir, "df_q11B_flags_alunos.parquet"), index=False)
df_q11B_resumo.to_parquet(os.path.join(out_dir, "df_q11B_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_heatmap.parquet', 'df_q9_resumo.csv', 'df_q9_resumo.parquet', 'df_q9_segmentos.csv', 'df_q9_segmentos.parquet', 'dummy_update.txt']


In [240]:
# ============================================================
# Pergunta 11-C — Insights e criatividade
# Tema: Alerta precoce do professor (IPP) antes da queda no desempenho
# Ideia:
#   Identificar casos onde o IPP está alto (alerta do professor),
#   mesmo quando o desempenho (INDE_final) ainda não está ruim.
# Critério:
#   - IPP alto = acima do 75º percentil (Q4)
#   - INDE_final "ok" = acima do 25º percentil (não está no fundo)
# Saídas:
#   - df_q11C_alerta_precoce_alunos  (lista de alunos em alerta precoce)
#   - df_q11C_resumo                (contagem por ano_base e fase)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

COL_ID = "ra_num"
COL_TXT = "RA"

needed = ["ano_base", "Fase_num", COL_ID, "IPP", "INDE_final"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-C:", missing)
else:
    for c in ["ano_base", "Fase_num", COL_ID, "IPP", "INDE_final"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    cols = [COL_ID, "ano_base", "Fase_num", "IPP", "INDE_final"]
    if COL_TXT in df.columns:
        cols = [COL_TXT] + cols

    tmp = df[cols].dropna(subset=["IPP", "INDE_final"]).copy()

    ipp_q4  = tmp["IPP"].quantile(0.75)
    inde_q1 = tmp["INDE_final"].quantile(0.25)

    tmp["alerta_precoce"] = ((tmp["IPP"] >= ipp_q4) & (tmp["INDE_final"] > inde_q1)).astype(int)

    df_q11C_alerta_precoce_alunos = tmp[tmp["alerta_precoce"] == 1].copy()

    df_q11C_resumo = (
        df_q11C_alerta_precoce_alunos
        .groupby(["ano_base", "Fase_num"], dropna=False)
        .size()
        .reset_index(name="qtd_alerta_precoce")
        .sort_values(["ano_base", "Fase_num"])
    )

    print("df_q11C_alerta_precoce_alunos -> linhas:", len(df_q11C_alerta_precoce_alunos))
    print(df_q11C_resumo.head(12))


df_q11C_alerta_precoce_alunos -> linhas: 564
    ano_base  Fase_num  qtd_alerta_precoce
0       2023         1                  68
1       2023         2                 109
2       2023         3                  15
3       2023         4                  35
4       2023         5                  17
5       2023         6                   7
6       2023         7                   4
7       2023      <NA>                  22
8       2024         1                  88
9       2024         2                  52
10      2024         3                  45
11      2024         4                  26


In [241]:
# ============================================================
# Pergunta 11-C — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11C_alerta_precoce_alunos
#   - df_q11C_resumo
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11C_alerta_precoce_alunos.to_csv(os.path.join(out_dir, "df_q11C_alerta_precoce_alunos.csv"), index=False, encoding="utf-8")
df_q11C_resumo.to_csv(os.path.join(out_dir, "df_q11C_resumo.csv"), index=False, encoding="utf-8")

df_q11C_alerta_precoce_alunos.to_parquet(os.path.join(out_dir, "df_q11C_alerta_precoce_alunos.parquet"), index=False)
df_q11C_resumo.to_parquet(os.path.join(out_dir, "df_q11C_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q11C_alerta_precoce_alunos.csv', 'df_q11C_alerta_precoce_alunos.parquet', 'df_q11C_resumo.csv', 'df_q11C_resumo.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_heatmap.parquet', 'df_q9_resumo.csv', 'df_q9_resumo.parquet', 'df_q9_segmentos.csv', 'df_q9_segmentos.parquet', 'dummy_updat

In [242]:
# ============================================================
# Pergunta 11-D — Insights e criatividade
# Tema: Volatilidade do desempenho (instabilidade) como sinal de risco
# Ideia:
#   Alguns alunos oscilam muito (sobe e desce), mesmo sem estar sempre no fundo.
#   Isso pode indicar problemas de constância, contexto familiar, saúde, etc.
# Método:
#   - Para cada aluno, calcular desvio-padrão do INDE_final ao longo dos anos
#   - Exigir pelo menos 2 anos de dados para entrar
# Saídas:
#   - df_q11D_volatilidade_alunos (aluno com métricas de estabilidade)
#   - df_q11D_resumo             (top volatilidade por ano_base e fase do último ano)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

COL_ID = "ra_num"
COL_TXT = "RA"

needed = ["ano_base", "Fase_num", COL_ID, "INDE_final"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-D:", missing)
else:
    for c in ["ano_base", "Fase_num", COL_ID, "INDE_final"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    cols = [COL_ID, "ano_base", "Fase_num", "INDE_final"]
    if COL_TXT in df.columns:
        cols = [COL_TXT] + cols

    tmp = df[cols].dropna(subset=["INDE_final"]).copy()
    tmp = tmp.sort_values([COL_ID, "ano_base"])

    agg = (
        tmp.groupby(COL_ID)
        .agg(
            anos=("ano_base", "nunique"),
            inde_media=("INDE_final", "mean"),
            inde_dp=("INDE_final", "std"),
            inde_min=("INDE_final", "min"),
            inde_max=("INDE_final", "max"),
            ano_ultimo=("ano_base", "max"),
        )
        .reset_index()
    )

    # só quem tem pelo menos 2 anos
    agg = agg[agg["anos"] >= 2].copy()

    # puxar fase do último ano (pra filtrar no Power BI)
    ult = tmp[tmp["ano_base"] == tmp.groupby(COL_ID)["ano_base"].transform("max")][[COL_ID, "Fase_num"]].copy()
    ult = ult.drop_duplicates(subset=[COL_ID])

    df_q11D_volatilidade_alunos = agg.merge(ult, on=COL_ID, how="left")

    # opcional: incluir RA texto
    if COL_TXT in df.columns:
        ra_map = df[[COL_ID, COL_TXT]].drop_duplicates(subset=[COL_ID])
        df_q11D_volatilidade_alunos = df_q11D_volatilidade_alunos.merge(ra_map, on=COL_ID, how="left")

    # resumo: top 50 mais voláteis por fase do último ano
    df_q11D_resumo = (
        df_q11D_volatilidade_alunos
        .sort_values(["inde_dp"], ascending=False)
        .groupby("Fase_num", dropna=False)
        .head(50)
        .reset_index(drop=True)
    )

    print("df_q11D_volatilidade_alunos -> linhas:", len(df_q11D_volatilidade_alunos))
    print(df_q11D_volatilidade_alunos.head(10))


df_q11D_volatilidade_alunos -> linhas: 825
   ra_num  anos  inde_media   inde_dp  inde_min  inde_max  ano_ultimo  \
0      19     2    7.687933  0.032433  7.665000  7.710867        2023   
1      27     2    7.522633  0.563753  7.124000  7.921267        2023   
2      32     2    7.043333  0.461976  6.716667  7.370000        2023   
3      38     2    8.679167  0.220853  8.523000  8.835333        2023   
4      39     2    7.371667  1.893161  6.033000  8.710333        2023   
5      40     3    7.157009  0.053662  7.098051  7.203000        2024   
6      42     3    7.057048  0.520587  6.720000  7.656629        2024   
7      43     2    5.974100  0.690278  5.486000  6.462200        2023   
8      44     2    5.900167  0.506524  5.542000  6.258333        2023   
9      45     3    8.212689  0.581591  7.591000  8.743476        2024   

   Fase_num     RA  
0         7  RA-19  
1         7  RA-27  
2         7  RA-32  
3         7  RA-38  
4         7  RA-39  
5         7  RA-40  
6     

In [243]:
# ============================================================
# Pergunta 11-D — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11D_volatilidade_alunos
#   - df_q11D_resumo
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11D_volatilidade_alunos.to_csv(os.path.join(out_dir, "df_q11D_volatilidade_alunos.csv"), index=False, encoding="utf-8")
df_q11D_resumo.to_csv(os.path.join(out_dir, "df_q11D_resumo.csv"), index=False, encoding="utf-8")

df_q11D_volatilidade_alunos.to_parquet(os.path.join(out_dir, "df_q11D_volatilidade_alunos.parquet"), index=False)
df_q11D_resumo.to_parquet(os.path.join(out_dir, "df_q11D_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q11C_alerta_precoce_alunos.csv', 'df_q11C_alerta_precoce_alunos.parquet', 'df_q11C_resumo.csv', 'df_q11C_resumo.parquet', 'df_q11D_resumo.csv', 'df_q11D_resumo.parquet', 'df_q11D_volatilidade_alunos.csv', 'df_q11D_volatilidade_alunos.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadrantes.parquet', 'df_q7_rank_ipv.csv', 'df_q7_rank_ipv.parquet', 'df_q8_heatmap.csv', 'df_q8_he

In [244]:
# ============================================================
# Pergunta 11-E — Insights e criatividade
# Tema: Ativo/Inativo (sinal de evasão) e relação com indicadores
# Ideia:
#   Ver se alunos Inativos tendem a ter:
#   - INDE_final mais baixo
#   - IPS mais baixo
#   - IEG mais baixo
# Saídas:
#   - df_q11E_base        (aluno a aluno com status e indicadores)
#   - df_q11E_resumo      (médias por status, ano_base e fase)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

COL_ID = "ra_num"
COL_TXT = "RA"

# coluna do status (no seu df ela existe como "Ativo/ Inativo")
COL_STATUS = "Ativo/ Inativo"

needed = ["ano_base", "Fase_num", COL_ID, COL_STATUS, "INDE_final", "IPS", "IEG"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-E:", missing)
else:
    # Tipos numéricos onde faz sentido
    for c in ["ano_base", "Fase_num", COL_ID, "INDE_final", "IPS", "IEG"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    cols = [COL_ID, "ano_base", "Fase_num", COL_STATUS, "INDE_final", "IPS", "IEG"]
    if COL_TXT in df.columns:
        cols = [COL_TXT] + cols

    tmp = df[cols].dropna(subset=[COL_STATUS]).copy()

    # Padronizar status (evita "Ativo ", "ativo", etc.)
    tmp["status"] = tmp[COL_STATUS].astype(str).str.strip().str.upper()

    df_q11E_base = tmp.copy()

    df_q11E_resumo = (
        df_q11E_base
        .groupby(["ano_base", "Fase_num", "status"], dropna=False)
        .agg(
            n=(COL_ID, "size"),
            inde_medio=("INDE_final", "mean"),
            ips_medio=("IPS", "mean"),
            ieg_medio=("IEG", "mean"),
        )
        .reset_index()
        .sort_values(["ano_base", "Fase_num", "status"])
    )

    print("df_q11E_base -> linhas:", len(df_q11E_base))
    print(df_q11E_resumo.head(12))


df_q11E_base -> linhas: 1156
   ano_base  Fase_num    status    n  inde_medio  ips_medio  ieg_medio
0      2024         1  CURSANDO  185    7.564359   6.968838   8.491705
1      2024         2  CURSANDO  185    7.371663   6.964784   7.960371
2      2024         3  CURSANDO  211    7.106355   7.170355   7.391482
3      2024         4  CURSANDO  115    7.296420   6.655826   7.947471
4      2024         5  CURSANDO  100    7.474831   6.740850   8.472958
5      2024         6  CURSANDO   25    7.843370   6.860000   8.795033
6      2024         7  CURSANDO   37    7.579202   7.296284   7.442178
7      2024         8  CURSANDO   64         NaN        NaN   0.000000
8      2024         9  CURSANDO   38         NaN        NaN   0.000000
9      2024      <NA>  CURSANDO  196    7.502120   6.259388   8.498762


In [245]:
# ============================================================
# Pergunta 11-E — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11E_base
#   - df_q11E_resumo
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11E_base.to_csv(os.path.join(out_dir, "df_q11E_base.csv"), index=False, encoding="utf-8")
df_q11E_resumo.to_csv(os.path.join(out_dir, "df_q11E_resumo.csv"), index=False, encoding="utf-8")

df_q11E_base.to_parquet(os.path.join(out_dir, "df_q11E_base.parquet"), index=False)
df_q11E_resumo.to_parquet(os.path.join(out_dir, "df_q11E_resumo.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q11C_alerta_precoce_alunos.csv', 'df_q11C_alerta_precoce_alunos.parquet', 'df_q11C_resumo.csv', 'df_q11C_resumo.parquet', 'df_q11D_resumo.csv', 'df_q11D_resumo.parquet', 'df_q11D_volatilidade_alunos.csv', 'df_q11D_volatilidade_alunos.parquet', 'df_q11E_base.csv', 'df_q11E_base.parquet', 'df_q11E_resumo.csv', 'df_q11E_resumo.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resumo.csv', 'df_q5_resumo.parquet', 'df_q6_base.csv', 'df_q6_base.parquet', 'df_q6_quadrantes.csv', 'df_q6_quadran

In [246]:
# ============================================================
# Pergunta 11-F — Insights e criatividade
# Tema: Diferenças por Turma/Escola (onde o desempenho está melhor ou pior)
# Ideia:
#   Comparar médias de INDE_final e indicadores (IDA, IEG, IPS, IPP, IAN)
#   por Turma e por Escola, sempre com n (tamanho da amostra).
# Saídas:
#   - df_q11F_por_turma   (médias e n por ano_base, fase e Turma)
#   - df_q11F_por_escola  (médias e n por ano_base, fase e Escola)
# ============================================================

import pandas as pd
import numpy as np

df = df_fase5.copy()

needed = ["ano_base", "Fase_num", "Turma", "Escola", "INDE_final", "IDA", "IEG", "IPS", "IPP", "IAN"]
missing = [c for c in needed if c not in df.columns]

if missing:
    print("ERRO: faltam colunas para a Pergunta 11-F:", missing)
else:
    # Tipos numéricos
    for c in ["ano_base", "Fase_num", "INDE_final", "IDA", "IEG", "IPS", "IPP", "IAN"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    base_cols = ["ano_base", "Fase_num", "Turma", "Escola", "INDE_final", "IDA", "IEG", "IPS", "IPP", "IAN"]
    tmp = df[base_cols].copy()

    # --- Por Turma ---
    df_q11F_por_turma = (
        tmp.dropna(subset=["Turma", "INDE_final"])
        .groupby(["ano_base", "Fase_num", "Turma"], dropna=False)
        .agg(
            n=("INDE_final", "size"),
            inde_medio=("INDE_final", "mean"),
            ida_media=("IDA", "mean"),
            ieg_medio=("IEG", "mean"),
            ips_medio=("IPS", "mean"),
            ipp_medio=("IPP", "mean"),
            ian_medio=("IAN", "mean"),
        )
        .reset_index()
        .sort_values(["ano_base", "Fase_num", "inde_medio"], ascending=[True, True, False])
    )

    # --- Por Escola ---
    df_q11F_por_escola = (
        tmp.dropna(subset=["Escola", "INDE_final"])
        .groupby(["ano_base", "Fase_num", "Escola"], dropna=False)
        .agg(
            n=("INDE_final", "size"),
            inde_medio=("INDE_final", "mean"),
            ida_media=("IDA", "mean"),
            ieg_medio=("IEG", "mean"),
            ips_medio=("IPS", "mean"),
            ipp_medio=("IPP", "mean"),
            ian_medio=("IAN", "mean"),
        )
        .reset_index()
        .sort_values(["ano_base", "Fase_num", "inde_medio"], ascending=[True, True, False])
    )

    print("df_q11F_por_turma -> linhas:", len(df_q11F_por_turma))
    print(df_q11F_por_turma.head(10))
    print("\ndf_q11F_por_escola -> linhas:", len(df_q11F_por_escola))
    print(df_q11F_por_escola.head(10))


df_q11F_por_turma -> linhas: 244
    ano_base  Fase_num Turma   n  inde_medio  ida_media  ieg_medio  ips_medio  \
22      2022         0     Z   9    7.929889   8.022222   8.588889   7.011111   
0       2022         0     A   6    7.849333   8.583333   8.300000   8.100000   
3       2022         0     D   9    7.749778   7.544444   9.066667   7.688889   
11      2022         0     L  11    7.694182   7.309091   9.300000   7.500000   
16      2022         0     R   8    7.692500   8.037500   7.950000   6.262500   
19      2022         0     U   9    7.675556   7.388889   8.644444   7.500000   
1       2022         0     B   6    7.646167   6.966667   9.000000   7.300000   
17      2022         0     S   7    7.578714   7.171429   8.085714   6.885714   
21      2022         0     Y  10    7.549800   8.140000   7.700000   7.250000   
12      2022         0     M   6    7.519500   7.766667   7.450000   7.083333   

    ipp_medio  ian_medio  
22        NaN   8.888889  
0         NaN   7.500

In [247]:
# ============================================================
# Pergunta 11-F — Exportação (CSV + Parquet)
# Arquivos:
#   - df_q11F_por_turma
#   - df_q11F_por_escola
# Saída em: /content/fase5/data/processed
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

df_q11F_por_turma.to_csv(os.path.join(out_dir, "df_q11F_por_turma.csv"), index=False, encoding="utf-8")
df_q11F_por_escola.to_csv(os.path.join(out_dir, "df_q11F_por_escola.csv"), index=False, encoding="utf-8")

df_q11F_por_turma.to_parquet(os.path.join(out_dir, "df_q11F_por_turma.parquet"), index=False)
df_q11F_por_escola.to_parquet(os.path.join(out_dir, "df_q11F_por_escola.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q11C_alerta_precoce_alunos.csv', 'df_q11C_alerta_precoce_alunos.parquet', 'df_q11C_resumo.csv', 'df_q11C_resumo.parquet', 'df_q11D_resumo.csv', 'df_q11D_resumo.parquet', 'df_q11D_volatilidade_alunos.csv', 'df_q11D_volatilidade_alunos.parquet', 'df_q11E_base.csv', 'df_q11E_base.parquet', 'df_q11E_resumo.csv', 'df_q11E_resumo.parquet', 'df_q11F_por_escola.csv', 'df_q11F_por_escola.parquet', 'df_q11F_por_turma.csv', 'df_q11F_por_turma.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv', 'df_q5_flags.parquet', 'df_q5_resum

In [248]:
# ============================================================
# Exportar o df_fase5 (base consolidada) para usar em outro notebook
# Saída em: /content/fase5/data/processed
# Arquivos:
#   - df_fase5.csv
#   - df_fase5.parquet
# ============================================================

import os

REPO_DIR = "/content/fase5"
out_dir = os.path.join(REPO_DIR, "data", "processed")
os.makedirs(out_dir, exist_ok=True)

# CSV (maior, mas universal)
df_fase5.to_csv(os.path.join(out_dir, "df_fase5.csv"), index=False, encoding="utf-8")

# Parquet (menor e mais rápido; preserva melhor os tipos)
df_fase5.to_parquet(os.path.join(out_dir, "df_fase5.parquet"), index=False)

print("Arquivos gerados em:", out_dir)
print(sorted(os.listdir(out_dir)))


Arquivos gerados em: /content/fase5/data/processed
['df_fase5.csv', 'df_fase5.parquet', 'df_ian.csv', 'df_ian.parquet', 'df_ida_ano.csv', 'df_ida_ano.parquet', 'df_ida_fase.csv', 'df_ida_fase.parquet', 'df_q11A_perfis_alunos.csv', 'df_q11A_perfis_alunos.parquet', 'df_q11A_perfis_resumo.csv', 'df_q11A_perfis_resumo.parquet', 'df_q11B_flags_alunos.csv', 'df_q11B_flags_alunos.parquet', 'df_q11B_resumo.csv', 'df_q11B_resumo.parquet', 'df_q11C_alerta_precoce_alunos.csv', 'df_q11C_alerta_precoce_alunos.parquet', 'df_q11C_resumo.csv', 'df_q11C_resumo.parquet', 'df_q11D_resumo.csv', 'df_q11D_resumo.parquet', 'df_q11D_volatilidade_alunos.csv', 'df_q11D_volatilidade_alunos.parquet', 'df_q11E_base.csv', 'df_q11E_base.parquet', 'df_q11E_resumo.csv', 'df_q11E_resumo.parquet', 'df_q11F_por_escola.csv', 'df_q11F_por_escola.parquet', 'df_q11F_por_turma.csv', 'df_q11F_por_turma.parquet', 'df_q4_quadrantes.csv', 'df_q4_quadrantes.parquet', 'df_q4_scatter.csv', 'df_q4_scatter.parquet', 'df_q5_flags.csv',

In [251]:
###############################################################################################################################################
#Criar ~/.netrc com o token do Secret "ITHUB_TOKEN" (sem expor o token)
#
# O que eu faço aqui:
# - Leio o token do Secret do Colab (ITHUB_TOKEN)
# - Se não achar, eu paro com erro claro (pra não gravar "None" no netrc)
# - Gravo ~/.netrc e protejo com permissão 600
###############################################################################################################################################
from google.colab import userdata
import os

token = userdata.get("ITHUB_TOKEN")   # <- do jeito que você definiu
user  = "tivanello"

if not token:
    raise ValueError('Secret "ITHUB_TOKEN" não encontrado ou vazio. Confira em Colab > Secrets.')

path = os.path.expanduser("~/.netrc")
with open(path, "w") as f:
    f.write(f"machine github.com\nlogin {user}\npassword {token}\n")
os.chmod(path, 0o600)

print("OK: ~/.netrc criado/atualizado (token não foi exibido).")


OK: ~/.netrc criado/atualizado (token não foi exibido).


In [252]:
###############################################################################################################################################
# Teste de acesso ao GitHub (sem expor token)
#
# O que eu faço aqui:
# - Chamo a API do GitHub para confirmar:
#   1) qual usuário o token representa
#   2) se o repo tivanello/fase5 é acessível
###############################################################################################################################################
import json, urllib.request
from google.colab import userdata

token = userdata.get("ITHUB_TOKEN")

def gh_get(url):
    req = urllib.request.Request(url)
    req.add_header("Authorization", f"token {token}")
    req.add_header("Accept", "application/vnd.github+json")
    with urllib.request.urlopen(req) as r:
        return json.loads(r.read().decode("utf-8"))

me = gh_get("https://api.github.com/user")
repo = gh_get("https://api.github.com/repos/tivanello/fase5")

print("Usuário autenticado:", me.get("login"))
print("Repo:", repo.get("full_name"))
print("Permissões:", repo.get("permissions"))


Usuário autenticado: tivanello
Repo: tivanello/fase5
Permissões: {'admin': True, 'maintain': True, 'push': True, 'triage': True, 'pull': True}


In [253]:
%%bash
###############################################################################################################################################
# BLOCO A — Clonar repo e entrar em /content/fase5
#
# O que eu faço aqui:
# - Vou para /content
# - Clono o repo se ele não existir
# - Entro no diretório do repo e confirmo caminho
###############################################################################################################################################
cd /content || exit 1
if [ ! -d "/content/fase5/.git" ]; then
  rm -rf /content/fase5
  git clone https://github.com/tivanello/fase5.git
fi
cd /content/fase5 || exit 1
pwd
git remote -v


/content/fase5
origin	https://github.com/tivanello/fase5.git (fetch)
origin	https://github.com/tivanello/fase5.git (push)


In [256]:
%%bash
###############################################################################################################################################
#  Versionar e enviar data/processed para o GitHub
#
# O que eu faço aqui:
# - Adiciono os arquivos de data/processed ao Git
# - Faço commit
# - Faço push para o GitHub
###############################################################################################################################################
cd /content/fase5 || exit 1
git add data/processed
git status
git commit -m "Adiciona todos os indicadores em data/processed" || true
# Pull latest changes from remote before pushing
git pull --rebase origin main
git push origin main


On branch main
Your branch and 'origin/main' have diverged,
and have 1 and 1 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

nothing to commit, working tree clean
On branch main
Your branch and 'origin/main' have diverged,
and have 1 and 1 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

nothing to commit, working tree clean


From https://github.com/tivanello/fase5
 * branch            main       -> FETCH_HEAD
Successfully rebased and updated refs/heads/main.
To https://github.com/tivanello/fase5.git
   1dab239..d89cf3a  main -> main
